In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os
import requests
import subprocess
import pandas as pd
import re
import glob
import csv

# 로우 데이터 확보를 위한 작업

1. hwp 파일 다운로드
2. csv 파일 변환
3. 한자 -> 한글 변환
4. 모든 csv 파일을 병합하여 df 만들기
---
5. 사람별로 컬럼 나누기 (◯를 기준으로 발언자가 나뉨)
6. 국회에 기록된 국회의원의 발언만 정리하기(국회의원 db 확보 완료)
7. 전체 국회 회기에 대해 국회의원 이름/정당(좌우)/발언으로 나뉘어진 df 생성

# csv 폴더 불러와서 파일 병합

In [2]:
# 현재 디렉토리에 있는 모든 csv 파일의 이름을 가져옵니다.
csv_files = glob.glob('D:\\MJ 데이터 공모전 로우 데이터\\국회의원 속기록(csv)\\15대_csv\\*.csv')

# 각 파일에서 첫 번째 행을 추출하여 저장할 빈 리스트를 생성합니다.
data_rows = []

# 각 파일에 대해 반복
for file_name in csv_files:
    # csv 파일을 읽습니다.
    # 파일에 헤더가 없음을 알리기 위해 header=None을 추가합니다.
    # 또한, 따옴표를 무시하도록 quoting=csv.QUOTE_NONE을 추가합니다.
    try:
        df = pd.read_csv(file_name, header=None, quoting=csv.QUOTE_NONE)
    except pd.errors.ParserError:
        print(f'Error parsing file {file_name}')
        continue

    # csv 파일이 비어있는지 확인합니다.
    if df.empty:
        print(f'{file_name} is empty')
        continue

    # 첫 번째 행만 추출하여 data_rows에 추가합니다.
    # 이때, 파일 이름을 첫 번째 열에 추가합니다.
    first_row = df.iloc[0].copy()  # 첫 번째 행이 데이터이므로 이를 사용합니다.
    first_row = first_row.tolist()  # 리스트로 변환합니다.
    first_row.insert(0, os.path.basename(file_name))  # 리스트의 첫 번째 위치에 파일 이름을 삽입합니다.
    data_rows.append(first_row)

# 모든 첫 번째 행을 하나의 데이터프레임으로 결합합니다.
merged_df = pd.DataFrame(data_rows)

In [3]:
merged_df['merged'] = merged_df.apply(lambda row: ' '.join(row.dropna().values.astype(str)), axis=1)
origin_df = merged_df[['merged']]

In [4]:
origin_df = merged_df.apply(lambda row: ' '.join(row.dropna().astype(str)), axis=1).to_frame()

In [ ]:
origin_df

## 여기에 각 회기별 특징적인 예외 단어들을 추가하면 됩니다.

In [5]:
# 17대에서 각 의원별 발언을 구분하는 ◯와 ○가 구분없이 사용되었으므로 수정했습니다.
origin_df = origin_df.applymap(lambda x: str(x).replace('○', '◯') if pd.notnull(x) else x)
origin_df = origin_df.applymap(lambda x: str(x).replace('○', '◯') if pd.notnull(x) else x)

In [6]:
# 16대에는 'ꠏ', 'ꠚ'(서로 다름)라는 특수문자가 의미없이 출현하므로 삭제했습니다. 읽어본 결과 특수문자 or 기호를 표현한 것 같습니다.
origin_df = origin_df.applymap(lambda x: str(x).replace('ꠏ', '') if pd.notnull(x) else x)
origin_df = origin_df.applymap(lambda x: str(x).replace('ꠚ', '') if pd.notnull(x) else x)

In [7]:
# 16대 파일에서 의미없는 단어를 찾아 삭제했습니다.
origin_df = origin_df.applymap(lambda x: str(x).replace('\u3000', ' ') if pd.notnull(x) else x)

In [8]:
# 한자 -> 한글 변환 과정에서 문제가 있는 단어를 변경했습니다.
origin_df = origin_df.applymap(lambda x: str(x).replace('립법', '입법') if pd.notnull(x) else x)

In [9]:
# 모든 회기에서 공통적으로 <그림>, <표>가 등장하므로 분석상 의미없는 단어를 삭제했습니다.
origin_df = origin_df.applymap(lambda x: str(x).replace('<그림>', '') if pd.notnull(x) else x)
origin_df = origin_df.applymap(lambda x: str(x).replace('<표>', '') if pd.notnull(x) else x)

In [10]:
# '따옴표가 의미없이 들어가서 이를 삭제했습니다.
origin_df = origin_df.applymap(lambda x: str(x).replace('\'', '') if pd.notnull(x) else x)
origin_df = origin_df.applymap(lambda x: str(x).replace('\"', '') if pd.notnull(x) else x)


- 16대에서 위원을 표현할 때 띄어쓰기가 제대로 되어있지 않아 추후 과정에서 문제가 생길 수 있습니다. 
- 앞뒤로 공백을 추가한 뒤, 공백이 두 번 중첩된 경우 이를 하나로 줄였습니다.

### 위와같이 진행할 경우, 위원장에서 문제가 생길 수 있기 때문에 위원장은 이 코드는 현 상태에서 사용하지 않는 것으로 합니다.
origin_df = origin_df.applymap(lambda x: str(x).replace('위원', ' 위원 ') if pd.notnull(x) else x)
origin_df = origin_df.applymap(lambda x: str(x).replace('  ', ' ') if pd.notnull(x) else x)

In [11]:
origin_df.iloc[96,0]

'국회회의록_15대_180회_5차_국방위원회.csv 0제180회-국방제5차(1996년7월25일)  제180회-국방제5차(1996년7월25일)(10시12분 개의)◯위원장 김영구 성원이 되었으므로 제5차 국방위원회를 개의하겠습니다.1. 업무현황보고(계속)가. 병무청◯위원장 김영구 의사일정 제1항 업무현황보고를 상정하겠습니다. 오늘은 병무청 소관 업무현황보고를 듣도록 하겠습니다. 병무청장 나오셔서 보고해 주시기 바랍니다.◯병무청장 송재환 병무청장 송재환입니다. 존경하는 김영구 국방위원장님  그리고 여러 국방위원님! 제15대 국회 국방위원회에서 병무청 업무현황보고를 드리게 된 것을 매우 영광스럽게 생각합니다. 그 동안 병무행정의 발전을 위하여 많은 관심을 가져 주시고 지도와 격려를 보내 주신데 대하여 충심으로 경의와 감사의 말씀을 드립니다. 병무행정은 국토방위를 위한 군 소요인력을 충원하는 행정으로 항상 국민들의 이목이 집중되어 있기 때문에 조그마한 실수나 사소한 비리도 용납될 수 없음을 누구보다도 우리 전직원은 잘 알고 있습니다. 따라서 정확  공정성은 병무행정의 기본이며 더 나아가 형평성 유지는 전국민의 신뢰를 증진하는 길이라는 신념을 가지고 단 한건의 불미스러운 일도 발생하지 않도록 심혈을 기울이고 있으며 유사시를 대비하여 병력동원 태세를 갖추는 데도 만전을 기하고 있습니다. 또한 병무행정은 안보적 측면에서 강제성을 띠고 있으나 여건의 변화에 순응하고 국민 불평해소와 사회기강의 파급효과를 고려하여 합리성과 타당성을 앞세워 최대한의 서비스를 제공함으로써 의무자로 하여금 자진하여 의무이행에 임할 수 있도록 전진하는 행정을 수행하는데 최선의 노력을 경주하겠습니다. 업무보고에 앞서 청 간부들이 인사를 드리도록 하겠습니다. 차장 이한구입니다. 기획관리관 김두성 부이사관입니다. 징모국장 한성남 이사관입니다. 동원소집국장 김태식 이사관입니다. 감사관 여춘욱 부이사관입니다. (간부 인사) 여러 위원님께서 양해해 주신다면 기획관리관으로 하여금 업무현황을 보고드리도록 하겠습니다.

## origin_df 파일 저장

In [13]:
file_path = 'D:\MJ 데이터 공모전 로우 데이터\\origins\\15_origin_df.csv'
origin_df.to_csv(file_path, index=False)

## orgin_df 불러오기

In [ ]:
# CSV 파일 경로
file_path = 'D:\\MJ 데이터 공모전 로우 데이터\\origins\\15_origin_df.csv'

# CSV 파일 불러오기
origin_df = pd.read_csv(file_path, encoding='utf-8', header=0)

In [14]:
origin_df

,0
0,국회회의록_15대_179회_1차_국회본회의.csv 0(제179회－제1차) (제17...
1,국회회의록_15대_179회_2차_국회본회의.csv 0(제179회－제2차) (제17...
2,국회회의록_15대_179회_3차_국회본회의.csv 0(제179회－제3차) (제17...
3,국회회의록_15대_179회_4차_국회본회의.csv 0(제179회－제4차) (제17...
4,국회회의록_15대_179회_5차_국회본회의.csv 0(제179회－제5차) (제17...
...,...
2892,국회회의록_15대_211회_1차_교육위원회.csv 0(제211회－교육제1차) (제...
2893,국회회의록_15대_211회_1차_국방위원회.csv 0(제211회-국방제1차) (...
2894,국회회의록_15대_211회_1차_농림해양수산위원회.csv 0 (제211회－농림해양수...
2895,국회회의록_15대_211회_1차_재정경제위원회.csv 0(제211회－재정경제제1차)...


## 과정 설명

1. origin_df를 불러온다.
    - origin_df가 망가지면 다시 불러와야 하므로 copied_df를 사용한다.
2. csv 파일 전체가 하나의 셀에 입력되어 있으므로 
    - 파일의 이름을 떼어내서 행의 카테고리로 사용하기 위해 간단한 전처리 작업을 거친다.
        - '회_'를 기준으로 스플릿하여 '차_'로 스플릿 할 때의 불상사를 막는다 ex) '자동차_', '소방차_'...
        - '파일의 제목을 헷갈리게 만드는 '국정조사', '국정감사', '개회식'은 삭제한다.
        - '(1)', '(2)' 등 문서의 개수를 나타내는 정보를 삭제한다.
        - 카테고리 뒤에 붙어있는 의미없는 공백, 탭 등의 개행문자를 rstrip()으로 삭제한다.
        - copied_df를 정렬하고 카테고리에 중복이 없는지 확인한다.
        - 확인이 완료되면 속기록 내부에서 발언자를 구분하는 '◯'를 사용하여 스플릿하고 grouped_df에 할당한다.

### origin_df 불러오기

In [15]:
copied_df = origin_df.copy()

In [16]:
copied_df

,0
0,국회회의록_15대_179회_1차_국회본회의.csv 0(제179회－제1차) (제17...
1,국회회의록_15대_179회_2차_국회본회의.csv 0(제179회－제2차) (제17...
2,국회회의록_15대_179회_3차_국회본회의.csv 0(제179회－제3차) (제17...
3,국회회의록_15대_179회_4차_국회본회의.csv 0(제179회－제4차) (제17...
4,국회회의록_15대_179회_5차_국회본회의.csv 0(제179회－제5차) (제17...
...,...
2892,국회회의록_15대_211회_1차_교육위원회.csv 0(제211회－교육제1차) (제...
2893,국회회의록_15대_211회_1차_국방위원회.csv 0(제211회-국방제1차) (...
2894,국회회의록_15대_211회_1차_농림해양수산위원회.csv 0 (제211회－농림해양수...
2895,국회회의록_15대_211회_1차_재정경제위원회.csv 0(제211회－재정경제제1차)...


### 카테고리 생성을 위한 전처리

In [17]:
copied_df = copied_df[0].str.replace('국정조사_', '').str.replace('국정감사_', '').str.replace('개회식_', '')

copied_df = pd.DataFrame(copied_df)

copied_df = copied_df[0].str.split('회_', expand=True)

copied_df = copied_df.drop([0, 1], axis=1)

copied_df = copied_df[2].str.split('.csv 0', expand=True)

copied_df[0] = copied_df[0].apply(lambda x: re.sub(r'\d{1,2}차_', '', x))

copied_df = pd.DataFrame(copied_df)

# 정규식 패턴과 숫자를 삭제하는 함수 정의
def remove_numbers_in_parentheses(text):
    import re
    pattern = r"\(\d+\)"
    return re.sub(pattern, '', str(text))

copied_df[0] = copied_df[0].apply(remove_numbers_in_parentheses)

copied_df = pd.DataFrame(copied_df)

copied_df[0] = copied_df[0].str.rstrip()

copied_df = copied_df.sort_values(by=copied_df.columns[0])

### 나눠진 카테고리의 중복이 없는지 확인

In [18]:
from collections import OrderedDict

data_list = copied_df.iloc[:, 0].tolist()
unique_data_list = list(OrderedDict.fromkeys(data_list).keys())

In [19]:
unique_data_list

['2002년월드컵등국제경기대회지원특별위원회',
 'IMF환란원인규명과경제위기진상조사를위한국정조사특별위원회',
 '건설교통위원회',
 '경제구조개혁및실업대책특별위원회',
 '과학기술정보통신위원회',
 '교육위원회',
 '국방위원회',
 '국제경기대회지원특별위원회',
 '국제경쟁력강화및경제제도개혁에관한특별위원회',
 '국회본회의',
 '국회운영위원회',
 '내무위원회',
 '농림수산위원회',
 '농림해양수산위원회',
 '농어민및도시영세민대책특별위원회',
 '문화관광위원회',
 '문화체육공보위원회',
 '법제사법위원회',
 '보건복지위원회',
 '산업자원위원회',
 '실업대책및경제구조개혁특별위원회',
 '언론문건진상규명을위한국정조사특별위원회',
 '여성특별위원회',
 '예산결산특별위원회',
 '윤리특별위원회',
 '재정경제위원회',
 '재해대책특별위원회',
 '정무위원회',
 '정치개혁입법특별위원회',
 '정치개혁입법특별위원회(1999.12.30부터)',
 '정치구조개혁입법특별위원회',
 '제15대국회의원총선거에있어서공정성시비국정조사특별위원회',
 '제도개선특별위원회',
 '통상산업위원회',
 '통신과학기술위원회',
 '통일외교통상위원회',
 '통일외무위원회',
 '한국조폐공사파업유도진상조사를위한국정조사특별위원회',
 '한보사건국정조사특별위원회',
 '행정위원회',
 '행정자치위원회',
 '환경노동위원회']

In [20]:
copied_df.iloc[518,1]

'1996년도국감-국방(1996년10월16일)  1996년도국감-국방(1996년10월16일)(10시25분 감사개시)◯위원장 김영구 지금으로부터 헌법 제61조  국회법 제127조  국정감사및조사에관한법률에 의하여 해병대사령부에 대한 1996년도 국정감사를 실시할 것을 선포합니다. 감사에 들어가기 전에 간단히 인사말씀 드리겠습니다. 오늘 우리 해병대사령부에 대한 국정감사를 실시함에 있어 전승에 결정적 기여를 하는 강력한 해병 전력을 유지 강화하기 위해 애쓰고 계신 전도봉 해병대사령관 이하 참모  장병 여러분들의 노고에 대해서 진심으로 치하의 말씀을 드립니다. 최근 잠수함을 이용한 무장공비 침투로 대규모 병력을 침투시키기 위한 정탐행위를 자행하고도 북한은 적반하장격으로 각종 수단을 동원해 대남보복위협발언을 자행하여 우리를 더욱 분노케 하고 있습니다. 최근 일련의 북한의 무력적 군사도발 행위는 우리로 하여금 북한의 실체를 다시 한번 정확히 판단하게 하였고  아울러 북의 어떠한 군사도발 행위에도 즉각 대응하기 위한 자주국방과 확고한 군사대비태세를 갖추어야 한다는 경각심을 우리 모두에게 일깨워 주었다고 생각합니다. 우리 해병대는 언제 어떠한 사태에도 대적 가능한 군사대비태세를 유지하기 위해 실전적 교육훈련을 강화하고  한미해병 협조체제를 공고히 하는 등 연합작전 능력향상과 함께 신앙적 해병정신으로 해병대의 전략적 가치를 높이기 위해 노력하고 계신 것으로 알고 있습니다. 아무쪼록 우리 해병대가 국민의 신뢰를 받는 해병대로 더욱더 발전해 주시기 바라며  이번 국정감사를 통해 해병대의 여러 가지 어려운 점과 발전 방향에 대해 진지한 논의가 있기를 바랍니다. 그러면 감사를 시작하도록 하겠습니다. 다음은 기관장의 선서가 있겠습니다. 증인 선서를 하기에 앞서 선서의 취지와 처벌규정 등에 대해서 말씀드리겠습니다. 선서를 하는 이유는 이번에 국회가 1996년도 국정감사를 실시함에 있어 증인으로부터 양심에 따라 숨김없이 사실대로 증언을 하겠다는 서약을 받기 위한 것입니다. 만일 증인이 

In [21]:
copied_df

,0,1
768,2002년월드컵등국제경기대회지원특별위원회,제183회-국제경기대회지원특별제5차(1997년2월17일) 제183회-국제경기대회지...
1038,2002년월드컵등국제경기대회지원특별위원회,제185회-국제경기대회지원특별제7차(1997년10월23일) 제185회-국제경기대회...
2466,2002년월드컵등국제경기대회지원특별위원회,제208회-국제경기대회지원특별제3차(1999년9월20일) 제208회-국제경기대회지...
2219,2002년월드컵등국제경기대회지원특별위원회,제203회-국제경기대회지원특별제2차(1999년4월26일) 제203회-국제경기대회지...
2486,2002년월드컵등국제경기대회지원특별위원회,제208회-국제경기대회지원특별제4차(1999년11월30일) 제208회-국제경기대회...
...,...,...
1000,환경노동위원회,제185회-환경노동제4차(1997년10월30일) 제185회-환경노동제4차(1997...
1712,환경노동위원회,제198회-환경노동제5차(1998년10월20일) 제198회-환경노동제5차(1998...
2505,환경노동위원회,제208회-환경노동제4차(1999년9월29일) 제208회-환경노동제4차(1999년...
663,환경노동위원회,1996년도국감-환경노동(1996년10월16일) 1996년도국감-환경노동(1996...


In [22]:
grouped_df = copied_df.groupby(copied_df.columns[0])[copied_df.columns[1]].apply(list).reset_index()

In [23]:
grouped_df[1] = grouped_df[1].apply(lambda lst: [item for sublist in lst for item in sublist.split('◯')] if isinstance(lst, list) else lst)

In [24]:
grouped_df.iloc[0, 1]

['제183회-국제경기대회지원특별제5차(1997년2월17일)  제183회-국제경기대회지원특별제5차(1997년2월17일)(16시16분 개의)',
 '위원장 신경식 의석을 정돈해 주시기 바랍니다. 성원이 되었으므로 제5차 2002년월드컵등국제경기대회지원특별위원회를 개의하겠습니다. 먼저 보고사항이 있겠습니다. 입법조사관 나와서 보고해 주시기 바랍니다.',
 '입법조사관 김기상 보고를 드리겠습니다.(보고사항은 끝에 실음)',
 '위원장 신경식 의사일정에 들어가기 전에 위원장의 신상에 관한 발언을 하고자 합니다. 존경하는 위원님 여러분  그 동안 우리 위원회는 위원님 여러분의 적극적인 참여와 의정활동에 힘입어 2002년월드컵축구대회지원법을 제정하는 등 국제경기대회지원을 위해 많은 활동을 해왔습니다. 얼마 전에 무주․전주유니버시아드대회가 성공적으로 끝났습니다마는 이제 부산동아시아경기대회가 금년 5월로 다가와 있고 그 외에도 99년 강원도동계아시아경기대회 등 각종 경기대회 지원을 위해서는 아직도 우리 특별위원회가 해야 할 일이 많습니다. 이러한 중요한 시기에 본인이 일신상의 이유로 위원장직을 수행할 수 없게 되어 오늘 위원회 전체회의를 통하여 사의를 표하고자 합니다. 국회법 제50조제4항의 규정에 따르면 위원장 궐위시 다수당의 간사께서 위원장직무대리로 위원회 주재를 할 수 있도록 되어 있으므로 신한국당 김기수 간사 위원님께서 사회를 맡아 주시기 바라며 저는 자리를 뜨겠습니다. 그 동안 정말로 감사했습니다. 저희 위원회가 자주 모이지는 못했습니다마는 모두 참 좋은 분들  마음으로 서로 믿고 도와주시고 격려해주신 덕분에 별 과오 없이 지내왔고 또 아까도 말씀드렸습니다마는 지금 우리 특위에서 해야 할 중요한 일들이 많은 이때에 같이 일을 하지 못하게 되어서 섭섭한 마음을 금치 못합니다마는  제가 위원회를 떠나더라도 우리 특위에 관한 일이라면 뒤에서나마 제가 할 수 있는 모든 지원을 열심히 하겠습니다. 그 동안 도와주시고 격려해 주신 데 대해서 다시 한번 진심으로 감사의 말씀을 

### 처음 두 단어 중 '위원'이 없는 원소 삭제해주는 코드

In [25]:
def remove_unwanted_elements(cell):
    if isinstance(cell, list):  # cell이 리스트 형인지 확인
        return [s for s in cell if any(word for word in re.split(' |,|\n', s)[:2] if "위원" in word)]
    else:
        return None

# 0번째 열 제외하고 적용
grouped_df.iloc[:, 1:] = grouped_df.iloc[:, 1:].applymap(remove_unwanted_elements)

### 조사대상 국회의원 1899명의 이름

In [28]:
names = ["이명박",	"강경식",	"강삼재",	"강성재",	"강용식",	"강재섭",	"강종희",	"강창희",	"강현욱",	"구천서",	"국창근",	"권기술",	"권노갑",	"권수창",	"권영자",	"권오을",	"권익현",	"권정달",	"권철현",	"길승흠",	"김경재 ",	"김고성",	"김광수",	"김광원",	"김근태",	"김기수",	"김기재",	"김기춘",	"김길환",	"김덕",	"김덕룡",	"김도언",	"김동욱",	"김동주",	"김명규",	"김명섭",	"김명윤",	"김무성",	"김문수",	"김민석",	"김범명",	"김병태",	"김복동",	"김봉호",	"김상우",	"김상현",	"김석원",	"김선길",	"김성곤",	"김수한",	"김영구",	"김영배",	"김영선",	"김영일",	"김영준",	"김영진",	"김영진",	"김영환",	"김옥두",	"김용갑",	"김용환",	"김운환",	"김원길",	"김윤환",	"김의재",	"김인곤",	"김인영",	"김일윤",	"김일주",	"김재천",	"김정수",	"김정숙",	"김종배",	"김종필",	"김종하",	"김종학",	"김종호",	"김중위",	"김진배",	"김진재",	"김찬우",	"김찬진",	"김철",	"김충일",	"김충조",	"김칠환",	"김태랑",	"김태식",	"김태호",	"김학원",	"김한길",	"김허남",	"김현욱",	"김형오",	"김호일",	"김홍신",	"김홍일",	"나오연",	"남경필",	"남궁진",	"남평우",	"노기태",	"노무현",	"노승우",	"류선호",	"류종수",	"맹형규",	"목요상",	"박관용",	"박광태",	"박구일",	"박근혜",	"박명환",	"박범진",	"박상규",	"박상복",	"박상천",	"박성범",	"박세직",	"박세환",	"박승국",	"박시균",	"박신원",	"박우병",	"박원홍",	"박정수",	"박정훈",	"박종근",	"박종우",	"박종웅",	"박주천",	"박준규",	"박찬주",	"박창달",	"박철언",	"박태준",	"박헌기",	"박희태",	"방용석",	"배종무",	"백남치",	"백승홍",	"변웅전",	"변정일",	"서상목",	"서석재",	"서정화",	"서정화",	"서청원",	"서한샘",	"서훈",	"설훈",	"손세일",	"손학규",	"송업교",	"송현섭",	"송훈석",	"신경식",	"신기남",	"신기하",	"신낙균",	"신상우",	"신영국",	"신영균",	"심정구",	"안동선",	"안상수",	"안상수",	"안재홍",	"안택수",	"양성철",	"양정규",	"어준선",	"오세응",	"오양순",	"오용운",	"오장섭",	"원유철",	"유용태",	"유재건",	"유흥수",	"윤원중",	"윤철상",	"윤한도",	"이강두",	"이강희",	"이건개",	"이경재",	"이국헌",	"이규정",	"이규택",	"이긍규",	"이길재",	"이동복",	"이동원",	"이만섭",	"이미경",	"이병희",	"이부영",	"이사철",	"이상득",	"이상만",	"이상배",	"이상수",	"이상현",	"이상희",	"이석현",	"이성재",	"이성호",	"이세기",	"이수인",	"이신범",	"이양희",	"이영일",	"이완구",	"이용삼",	"이우재",	"이웅희",	"이원범",	"이원복",	"이윤성",	"이윤수",	"이응선",	"이의익",	"이인구",	"이재명",	"이재선",	"이재오",	"이재창",	"이정무",	"이중재",	"이찬진",	"이태섭",	"이택석",	"이한동",	"이해구",	"이해봉",	"이해찬",	"이협",	"이형배",	"이홍구",	"이회창",	"이훈평",	"임복진",	"임인배",	"임진출",	"임채정",	"장성원",	"장영달",	"장영철",	"장을병",	"장재식",	"전석홍",	"전용원",	"정균환",	"정동영",	"정동채",	"정몽준",	"정문화",	"정상구",	"정상천",	"정석모",	"정세균",	"정영훈",	"정우택",	"정의화",	"정일영",	"정재문",	"정재철",	"정창화",	"정한용",	"정형근",	"정호선",	"정희경",	"제정구",	"조성준",	"조세형",	"조순",	"조순승",	"조순형",	"조영재",	"조웅규",	"조익현",	"조중연",	"조진형",	"조찬형",	"조철구",	"조한천",	"조홍규",	"주양자",	"주진우",	"지대섭",	"차수명",	"채영석",	"천용택",	"천정배",	"최병렬",	"최선영",	"최연희",	"최재승",	"최형우",	"최희준",	"추미애",	"하경근",	"하순봉",	"한광옥",	"한승수",	"한영수",	"한영애",	"한이헌",	"한호선",	"한화갑",	"함석재",	"함종한",	"허남훈",	"허대범",	"허화평",	"현경대",	"홍문종",	"홍사덕",	"홍인길",	"황규선",	"황낙주",	"황병태",	"황성균",	"황승민",	"황우여",	"황학수",	"강봉균",	"강삼재",	"강성구",	"강숙자",	"강신성일",	"강운태",	"강인섭",	"강재섭",	"강창성",	"강창희",	"강현욱",	"고진부",	"고흥길",	"구종태",	"권기술",	"권영세",	"권오을",	"권철현",	"권태망",	"김경재 ",	"김경천",	"김광원",	"김근태",	"김기배",	"김기재",	"김기춘",	"김덕규",	"김덕룡",	"김덕배",	"김동욱",	"김락기",	"김만제",	"김명섭",	"김무성",	"김문수",	"김민석",	"김방림",	"김병호",	"김부겸",	"김상현",	"김성순",	"김성조",	"김성호",	"김영구",	"김영구",	"김영선",	"김영일",	"김영진",	"김영춘",	"김영환",	"김옥두",	"김용갑",	"김용균",	"김용학",	"김용환",	"김운용",	"김원기",	"김원길",	"김원웅",	"김일윤",	"김정부",	"김정숙",	"김종필",	"김종하",	"김종호",	"김진재",	"김찬우",	"김충조",	"김태식",	"김태호",	"김택기",	"김학송",	"김학원",	"김한길",	"김형오",	"김홍신",	"김홍일",	"김화중",	"김황식",	"김효석",	"김희선",	"나오연",	"남경필",	"남궁석",	"도종이",	"류시민",	"맹형규",	"목요상",	"문석호",	"문희상",	"민봉기",	"박경섭",	"박관용",	"박광태",	"박근혜",	"박금자",	"박명환",	"박병석",	"박병윤",	"박상규",	"박상천",	"박상희",	"박세환",	"박승국",	"박시균",	"박양수",	"박원홍",	"박인상",	"박재욱",	"박종근",	"박종완",	"박종우",	"박종웅",	"박종희",	"박주선",	"박주천",	"박진",	"박창달",	"박헌기",	"박혁규",	"박희태",	"배기선",	"배기운",	"백승홍",	"변웅전",	"서병수",	"서상섭",	"서영훈",	"서정화",	"서청원",	"설송웅",	"설훈",	"손태인",	"손학규",	"손희정",	"송광호",	"송병대",	"송석찬",	"송영길",	"송영진",	"송훈석",	"신경식",	"신계륜",	"신기남",	"신영국",	"신영균",	"신현태",	"심규섭",	"심규철",	"심재권",	"심재철",	"안경률",	"안대륜",	"안동선",	"안상수",	"안상현",	"안영근",	"안택수",	"안희옥",	"양승부",	"양정규",	"엄호성",	"오경훈",	"오세훈",	"오영식",	"오장섭",	"원유철",	"원철희",	"원희룡",	"유삼남",	"유용태",	"유재건",	"유재규",	"유한열",	"유흥수",	"윤경식",	"윤두환",	"윤여준",	"윤영탁",	"윤철상",	"윤한도",	"이강두",	"이강래",	"이경재",	"이규택",	"이근진",	"이낙연",	"이만섭",	"이만재",	"이미경",	"이방호",	"이병석",	"이부영",	"이상득",	"이상배",	"이상수",	"이상희",	"이성헌",	"이승철",	"이양희",	"이연숙",	"이완구",	"이용삼",	"이우재",	"이원성",	"이원창",	"이원형",	"이윤성",	"이윤수",	"이인기",	"이인제",	"이재선",	"이재오",	"이재정",	"이재창",	"이정일",	"이종걸",	"이종성",	"이주영",	"이창복",	"이한구",	"이한동",	"이해구",	"이해봉",	"이해찬",	"이협",	"이호웅",	"이회창",	"이훈평",	"이희규",	"임인배",	"임종석",	"임진출",	"임채정",	"임태희",	"장광근",	"장성원",	"장영달",	"장재식",	"장태완",	"전갑길",	"전용원",	"전용학",	"전재희",	"정갑윤",	"정균환",	"정대철",	"정동영",	"정동채",	"정몽준",	"정문화",	"정범구",	"정병국",	"정세균",	"정우택",	"정의화",	"정장선",	"정진석",	"정창화",	"정철기",	"정형근",	"조배숙",	"조부영",	"조성준",	"조순형",	"조웅규",	"조재환",	"조정무",	"조한천",	"조희욱",	"주진우",	"천용택",	"천정배",	"최돈웅",	"최명헌",	"최병국",	"최병렬",	"최선영",	"최연희",	"최영희",	"최용규",	"최재승",	"추미애",	"하순봉",	"한명숙",	"한승수",	"한충수",	"한화갑",	"함석재",	"함승희",	"허운나",	"허태열",	"현경대",	"현승일",	"홍문종",	"홍사덕",	"홍재형",	"홍준표",	"황승민",	"황우여",	"황창주",	"강기갑",	"강기정",	"강길부",	"강봉균",	"강성종",	"강재섭",	"강창일",	"강혜숙",	"고경화",	"고조흥",	"고진화",	"고흥길",	"고희선",	"공성진",	"곽성문",	"구논회",	"권경석",	"권선택",	"권영길",	"권영세",	"권오을",	"권철현",	"김광원",	"김교흥",	"김근태",	"김기춘",	"김기현",	"김낙성",	"김낙순",	"김덕규",	"김덕룡",	"김동철",	"김명자",	"김명주",	"김무성",	"김문수",	"김병호",	"김부겸",	"김석준",	"김선미",	"김성곤",	"김성조",	"김송자",	"김애실",	"김양수",	"김영대",	"김영덕",	"김영선",	"김영숙",	"김영주",	"김영춘",	"김용갑",	"김우남",	"김원기",	"김원웅",	"김재경",	"김재원",	"김재윤",	"김재홍",	"김정권",	"김정부",	"김정훈",	"김종률",	"김종인",	"김진표",	"김춘진",	"김충환",	"김태년",	"김태홍",	"김태환",	"김학송",	"김학원",	"김한길",	"김혁규",	"김현미",	"김형오",	"김형주",	"김홍업",	"김홍일",	"김효석",	"김희선",	"김희정",	"나경원",	"남경필",	"노영민",	"노웅래",	"노현송",	"노회찬",	"단병호",	"류근찬",	"류선호",	"류시민",	"맹형규",	"문병호",	"문석호",	"문학진",	"문희",	"문희상",	"민병두",	"박계동",	"박근혜",	"박기춘",	"박명광",	"박병석",	"박상돈",	"박성범",	"박세일",	"박세환",	"박순자",	"박승환",	"박영선",	"박재완",	"박종근",	"박진",	"박찬석",	"박찬숙",	"박형준",	"박홍수",	"박희태",	"배기선",	"배일도",	"백원우",	"변재일",	"서갑원",	"서병수",	"서상기",	"서재관",	"서혜석",	"선병렬",	"손봉숙",	"송영길",	"송영선",	"신계륜",	"신국환",	"신기남",	"신명",	"신상진",	"신중식",	"신학용",	"심대평",	"심상정",	"심재덕",	"심재엽",	"심재철",	"안경률",	"안명옥",	"안민석",	"안병엽",	"안상수",	"안영근",	"안택수",	"안홍준",	"양승조",	"양형일",	"엄호성",	"염동연",	"오영식",	"오제세",	"우상호",	"우원식",	"우윤근",	"우제창",	"우제항",	"원혜영",	"원희룡",	"유기준",	"유기홍",	"유승민",	"유승희",	"유인태",	"유재건",	"유정복",	"유필우",	"윤건영",	"윤원호",	"윤호중",	"이강두",	"이강래",	"이경숙",	"이경재",	"이계경",	"이계안",	"이계진",	"이광재",	"이광철",	"이군현",	"이규택",	"이근식",	"이기우",	"이낙연",	"이명규",	"이목희",	"이미경",	"이방호",	"이병석",	"이상경",	"이상득",	"이상민",	"이상배",	"이상열",	"이석현",	"이성구",	"이성권",	"이승희",	"이시종",	"이영순",	"이영호",	"이용희",	"이원복",	"이원영",	"이윤성",	"이은영",	"이인기",	"이인영",	"이인제",	"이재오",	"이재웅",	"이재창",	"이정일",	"이종걸",	"이종구",	"이주영",	"이주호",	"이진구",	"이한구",	"이해봉",	"이해찬",	"이혜훈",	"이호웅",	"이화영",	"임인배",	"임종석",	"임종인",	"임채정",	"임태희",	"임해규",	"장경수",	"장복심",	"장영달",	"장윤석",	"장향숙",	"전병헌",	"전여옥",	"전재희",	"정갑윤",	"정덕구",	"정동채",	"정두언",	"정몽준",	"정문헌",	"정병국",	"정봉주",	"정성호",	"정세균",	"정의용",	"정의화",	"정장선",	"정종복",	"정진석",	"정진섭",	"정청래",	"정형근",	"정화원",	"정희수",	"제종길",	"조경태",	"조배숙",	"조성래",	"조성태",	"조순형",	"조일현",	"조정식",	"주성영",	"주승용",	"주호영",	"지병문",	"진수희",	"진영",	"차명진",	"채수찬",	"채일병",	"천영세",	"천정배",	"최경환",	"최구식",	"최규성",	"최규식",	"최병국",	"최성",	"최순영",	"최연희",	"최용규",	"최인기",	"최재성",	"최재천",	"최철국",	"한광원",	"한명숙",	"한병도",	"한선교",	"한화갑",	"허천",	"허태열",	"현경병",	"현애자",	"홍문표",	"홍미영",	"홍재형",	"홍준표",	"홍창선",	"황우여",	"황진하",	"강기갑",	"강기정",	"강길부",	"강명순",	"강봉균",	"강석호",	"강성종",	"강성천",	"강승규",	"강용석",	"강운태",	"강창일",	"고승덕",	"고흥길",	"공성진",	"곽정숙",	"구상찬",	"권경석",	"권선택",	"권성동",	"권영길",	"권영세",	"권영진",	"권택기",	"김광림",	"김금래",	"김기현",	"김낙성",	"김동성",	"김동철",	"김무성",	"김부겸",	"김상희",	"김선동",	"김선동",	"김성곤",	"김성동",	"김성수",	"김성순",	"김성식",	"김성조",	"김성태",	"김성회",	"김세연",	"김소남",	"김영록",	"김영선",	"김영우",	"김영진",	"김영환",	"김옥이",	"김용구",	"김용태",	"김우남",	"김유정",	"김을동",	"김일윤",	"김장수",	"김재경",	"김재균",	"김재윤",	"김정",	"김정권",	"김정훈",	"김종률",	"김진애",	"김진표",	"김창수",	"김춘진",	"김충조",	"김충환",	"김태원",	"김태호",	"김태환",	"김학송",	"김학용",	"김학재",	"김형오",	"김혜성",	"김호연",	"김효석",	"김효재",	"김희철",	"나경원",	"나성린",	"남경필",	"노영민",	"노철래",	"류근찬",	"류선호",	"문학진",	"문희상",	"박근혜",	"박기춘",	"박대해",	"박민식",	"박병석",	"박보환",	"박상돈",	"박상은",	"박상천",	"박선숙",	"박선영",	"박순자",	"박영선",	"박영아",	"박우순",	"박은수",	"박종근",	"박주선",	"박준선",	"박지원",	"박진",	"박희태",	"배영식",	"배은희",	"백성운",	"백원우",	"백재현",	"변웅전",	"변재일",	"서갑원",	"서병수",	"서상기",	"서종표",	"선경식",	"성윤환",	"손범규",	"손숙미",	"손학규",	"송광호",	"송민순",	"송영길",	"송영선",	"송훈석",	"신건",	"신낙균",	"신상진",	"신성범",	"신영수",	"신지호",	"신학용",	"심대평",	"심재철",	"안경률",	"안규백",	"안민석",	"안상수",	"안형환",	"안홍준",	"안효대",	"양승조",	"여상규",	"오제세",	"우윤근",	"우제창",	"원유철",	"원혜영",	"원희룡",	"원희목",	"유기준",	"유성엽",	"유승민",	"유원일",	"유일호",	"유재중",	"유정복",	"유정현",	"윤상일",	"윤상현",	"윤석용",	"윤영",	"윤진식",	"이강래",	"이경재",	"이계진",	"이광재",	"이군현",	"이낙연",	"이달곤",	"이두아",	"이명규",	"이명수",	"이미경",	"이범관",	"이범래",	"이병석",	"이사철",	"이상권",	"이상득",	"이상민",	"이석현",	"이성남",	"이성헌",	"이시종",	"이애주",	"이영애",	"이영애",	"이용경",	"이용섭",	"이용희",	"이윤석",	"이윤성",	"이은재",	"이인기",	"이인제",	"이재선",	"이재오",	"이정선",	"이정현",	"이정희",	"이종걸",	"이종구",	"이종혁",	"이주영",	"이진복",	"이진삼",	"이찬열",	"이철우",	"이춘석",	"이춘식",	"이학재",	"이한구",	"이한성",	"이해봉",	"이혜훈",	"이화수",	"이회창",	"임동규",	"임두성",	"임영호",	"임태희",	"임해규",	"장광근",	"장병완",	"장세환",	"장윤석",	"장제원",	"전병헌",	"전여옥",	"전재희",	"전현희",	"전혜숙",	"정갑윤",	"정동영",	"정두언",	"정몽준",	"정미경",	"정범구",	"정병국",	"정세균",	"정수성",	"정양석",	"정영희",	"정옥임",	"정의화",	"정장선",	"정진석",	"정진섭",	"정태근",	"정하균",	"정해걸",	"정희수",	"조경태",	"조문환",	"조배숙",	"조순형",	"조승수",	"조영택",	"조원진",	"조윤선",	"조전혁",	"조정식",	"조진래",	"조진형",	"조해진",	"주광덕",	"주성영",	"주승용",	"주호영",	"진성호",	"진수희",	"진영",	"차명진",	"천정배",	"최경환",	"최경희",	"최구식",	"최규성",	"최규식",	"최문순",	"최병국",	"최연희",	"최영희",	"최인기",	"최재성",	"최종원",	"최철국",	"추미애",	"한기호",	"한선교",	"허원제",	"허천",	"허태열",	"현경병",	"현기환",	"홍사덕",	"홍영표",	"홍일표",	"홍재형",	"홍정욱",	"홍준표",	"홍희덕",	"황영철",	"황우여",	"황진하",	"강기윤",	"강기정",	"강길부",	"강동원",	"강석호",	"강석훈",	"강은희",	"강창일",	"강창희",	"경대수",	"고희선",	"권성동",	"권은희",	"권은희",	"길정우",	"김경협",	"김관영",	"김광림",	"김광진",	"김기선",	"김기식",	"김기준",	"김기현",	"김도읍",	"김동완",	"김동철",	"김명연",	"김무성",	"김미희",	"김민기",	"김상민",	"김상훈",	"김상희",	"김선동",	"김성곤",	"김성주",	"김성찬",	"김성태",	"김세연",	"김승남",	"김영록",	"김영우",	"김영주",	"김영환",	"김용남",	"김용익",	"김용태",	"김우남",	"김윤덕",	"김을동",	"김장실",	"김재경",	"김재연",	"김재원",	"김재윤",	"김정록",	"김정훈",	"김제남",	"김제식",	"김종태",	"김종훈",	"김진태",	"김진표",	"김춘진",	"김태년",	"김태원",	"김태호",	"김태환",	"김태흠",	"김학용",	"김한길",	"김한표",	"김현",	"김현미",	"김현숙",	"김회선",	"김희국",	"김희정",	"나경원",	"나성린",	"남경필",	"남인순",	"노영민",	"노웅래",	"노철래",	"노회찬",	"도종환",	"류성걸",	"류지영",	"문대성",	"문병호",	"문재인",	"문정림",	"문희상",	"민병두",	"민병주",	"민현주",	"민홍철",	"박광온",	"박근혜",	"박기춘",	"박남춘",	"박대동",	"박대출",	"박덕흠",	"박맹우",	"박명재",	"박민수",	"박민식",	"박범계",	"박병석",	"박상은",	"박성호",	"박성효",	"박수현",	"박영선",	"박완주",	"박원석",	"박윤옥",	"박인숙",	"박주선",	"박지원",	"박창식",	"박혜자",	"박홍근",	"배기운",	"배덕광",	"배재정",	"백군기",	"백재현",	"변재일",	"부좌현",	"서기호",	"서병수",	"서상기",	"서영교",	"서용교",	"서청원",	"설훈",	"손인춘",	"송광호",	"송영근",	"송호창",	"신경림",	"신경민",	"신계륜",	"신기남",	"신동우",	"신문식",	"신상진",	"신성범",	"신의진",	"신정훈",	"신학용",	"심상정",	"심윤조",	"심재권",	"심재철",	"심학봉",	"안규백",	"안민석",	"안상수",	"안종범",	"안철수",	"안홍준",	"안효대",	"양승조",	"양창영",	"여상규",	"염동열",	"오병윤",	"오신환",	"오영식",	"오제세",	"우상호",	"우원식",	"우윤근",	"원유철",	"원혜영",	"유기준",	"유기홍",	"유대운",	"유성엽",	"유승민",	"유승우",	"유승희",	"유은혜",	"유의동",	"유인태",	"유일호",	"유재중",	"유정복",	"윤관석",	"윤금순",	"윤명희",	"윤상현",	"윤영석",	"윤재옥",	"윤진식",	"윤호중",	"윤후덕",	"은수미",	"이강후",	"이개호",	"이군현",	"이낙연",	"이노근",	"이만우",	"이명수",	"이목희",	"이미경",	"이병석",	"이상규",	"이상민",	"이상일",	"이상직",	"이석기",	"이석현",	"이언주",	"이에리사",	"이완구",	"이완영",	"이용섭",	"이우현",	"이운룡",	"이원욱",	"이윤석",	"이이재",	"이인영",	"이인제",	"이자스민",	"이장우",	"이재영",	"이재오",	"이정현",	"이종걸",	"이종배",	"이종진",	"이종훈",	"이주영",	"이진복",	"이찬열",	"이채익",	"이철우",	"이춘석",	"이학영",	"이학재",	"이한구",	"이한성",	"이해찬",	"이헌승",	"이현재",	"인재근",	"임내현",	"임수경",	"장병완",	"장윤석",	"장정은",	"장하나",	"전병헌",	"전순옥",	"전정희",	"전하진",	"전해철",	"정갑윤",	"정두언",	"정몽준",	"정문헌",	"정미경",	"정병국",	"정성호",	"정세균",	"정수성",	"정용기",	"정우택",	"정윤숙",	"정의화",	"정진후",	"정청래",	"정호준",	"정희수",	"조경태",	"조명철",	"조원진",	"조정식",	"조해진",	"조현룡",	"주승용",	"주영순",	"주호영",	"진선미",	"진성준",	"진영",	"천정배",	"최경환",	"최규성",	"최동익",	"최민희",	"최봉홍",	"최원식",	"최재성",	"최재천",	"추미애",	"하태경",	"한기호",	"한명숙",	"한선교",	"한정애",	"함진규",	"홍문종",	"홍문표",	"홍영표",	"홍의락",	"홍익표",	"홍일표",	"홍종학",	"홍지만",	"홍철호",	"황영철",	"황우여",	"황인자",	"황주홍",	"황진하",	"강길부",	"강병원",	"강석진",	"강석호",	"강창일",	"강효상",	"강훈식",	"경대수",	"고용진",	"곽대훈",	"곽상도",	"권미혁",	"권석창",	"권성동",	"권은희",	"권칠승",	"금태섭",	"기동민",	"김경수",	"김경진",	"김경협",	"김관영",	"김광림",	"김광수",	"김규환",	"김기선",	"김도읍",	"김동철",	"김두관",	"김명연",	"김무성",	"김민기",	"김병관",	"김병기",	"김병욱",	"김부겸",	"김삼화",	"김상훈",	"김상희",	"김석기",	"김선동",	"김성수",	"김성식",	"김성원",	"김성찬",	"김성태",	"김성태",	"김성환",	"김세연",	"김수민",	"김순례",	"김승희",	"김영우",	"김영주",	"김영진",	"김영춘",	"김영호",	"김용태",	"김재경",	"김재원",	"김정우",	"김정재",	"김정호",	"김정훈",	"김종대",	"김종민",	"김종석",	"김종인",	"김종회",	"김종훈",	"김중로",	"김진태",	"김진표",	"김철민",	"김태년",	"김태흠",	"김학용",	"김한정",	"김한표",	"김해영",	"김현권",	"김현미",	"김현아",	"나경원",	"남인순",	"노웅래",	"노회찬",	"도종환",	"맹성규",	"문미옥",	"문진국",	"문희상",	"민경욱",	"민병두",	"민홍철",	"박경미",	"박광온",	"박남춘",	"박대출",	"박덕흠",	"박맹우",	"박명재",	"박범계",	"박병석",	"박선숙",	"박성중",	"박순자",	"박영선",	"박완수",	"박완주",	"박용진",	"박인숙",	"박재호",	"박정",	"박주민",	"박주선",	"박주현",	"박준영",	"박지원",	"박찬대",	"박찬우",	"박홍근",	"배덕광",	"백승주",	"백재현",	"백혜련",	"변재일",	"서삼석",	"서영교",	"서청원",	"서형수",	"설훈",	"성일종",	"소병훈",	"손금주",	"손혜원",	"송갑석",	"송기석",	"송기헌",	"송석준",	"송언석",	"송영길",	"송옥주",	"송희경",	"신경민",	"신동근",	"신보라",	"신상진",	"신용현",	"신창현",	"심기준",	"심상정",	"심재권",	"심재철",	"안규백",	"안민석",	"안상수",	"안철수",	"안호영",	"양승조",	"어기구",	"엄용수",	"여상규",	"여영국",	"염동열",	"오세정",	"오신환",	"오영훈",	"오제세",	"우상호",	"우원식",	"원유철",	"원혜영",	"위성곤",	"유기준",	"유동수",	"유민봉",	"유성엽",	"유승민",	"유승희",	"유은혜",	"유의동",	"유재중",	"윤관석",	"윤상직",	"윤상현",	"윤소하",	"윤영석",	"윤영일",	"윤일규",	"윤재옥",	"윤종오",	"윤종필",	"윤준호",	"윤한홍",	"윤호중",	"윤후덕",	"이개호",	"이군현",	"이규희",	"이동섭",	"이만희",	"이명수",	"이상돈",	"이상민",	"이상헌",	"이석현",	"이수혁",	"이양수",	"이언주",	"이완영",	"이용득",	"이용주",	"이용호",	"이우현",	"이원욱",	"이은권",	"이은재",	"이인영",	"이장우",	"이재정",	"이정미",	"이정현",	"이종걸",	"이종구",	"이종명",	"이종배",	"이주영",	"이진복",	"이찬열",	"이채익",	"이철규",	"이철우",	"이철희",	"이춘석",	"이태규",	"이학영",	"이학재",	"이해찬",	"이헌승",	"이현재",	"이혜훈",	"이후삼",	"이훈",	"인재근",	"임이자",	"임재훈",	"임종성",	"장병완",	"장석춘",	"장정숙",	"장제원",	"전재수",	"전해철",	"전현희",	"전혜숙",	"전희경",	"정갑윤",	"정동영",	"정병국",	"정성호",	"정세균",	"정양석",	"정용기",	"정우택",	"정운천",	"정유섭",	"정은혜",	"정인화",	"정재호",	"정점식",	"정종섭",	"정진석",	"정춘숙",	"정태옥",	"제윤경",	"조경태",	"조배숙",	"조승래",	"조원진",	"조응천",	"조정식",	"조훈현",	"주광덕",	"주승용",	"주호영",	"지상욱",	"진선미",	"진영",	"채이배",	"천정배",	"최경환",	"최경환",	"최교일",	"최도자",	"최명길",	"최연혜",	"최운열",	"최인호",	"최재성",	"추경호",	"추미애",	"추혜선",	"표창원",	"하태경",	"한선교",	"한정애",	"함진규",	"허윤정",	"홍문종",	"홍문표",	"홍영표",	"홍의락",	"홍익표",	"홍일표",	"홍철호",	"황영철",	"황주홍",	"황희",]

In [27]:
len(names)

1899

### names(국회의원 이름 포함된 리스트)에 이름이 없는 원소 삭제해주는 코드

- 몇 개 삭제했는지 확인 가능
- 처음엔 코드가 불안정해서 삭제되어버리는 셀이 있어 추가한 기능인데, 지금은 필요 없음

In [29]:
def remove_elements_without_names_and_count(cell):
    if isinstance(cell, list):  # cell이 리스트 형인지 확인
        original_length = len(cell)
        cell = [s for s in cell if any(word for word in re.split(' |,|\n', s)[:2] if word in names)]
        deleted_elements = original_length - len(cell)
        return cell, deleted_elements
    else:
        return None, 0

# 1. 결과를 저장할 데이터프레임 초기화
deleted_cells_df = pd.DataFrame(index=grouped_df.index, columns=grouped_df.columns[1:], data=0)

# 2. 0번째 열 제외하고 적용
for i in range(1, len(grouped_df.columns)):
    result = grouped_df.iloc[:, i].map(remove_elements_without_names_and_count)
    grouped_df.iloc[:, i] = result.apply(lambda x: x[0])
    deleted_cells_df.iloc[:, i - 1] = result.apply(lambda x: x[1])

# 3. 각 행에서 삭제된 셀의 총 개수 계산
deleted_cells_df['total_deleted_cells'] = deleted_cells_df.sum(axis=1)

# 4. 결과 출력
print(deleted_cells_df.iloc[:,1])
print(max(deleted_cells_df.iloc[:,1]))

0      187
1      106
2     1206
3      120
4      293
5      803
6      485
7       13
8       25
9      499
10     636
11    2338
12      19
13    1149
14      25
15     971
16     510
17    1821
18     712
19    1753
20       9
21       8
22     166
23    1158
24      27
25    1009
26      44
27    2494
28      93
29       3
30      15
31      78
32      64
33    1439
34     254
35     485
36     317
37     119
38    2147
39     575
40    1965
41    1836
Name: total_deleted_cells, dtype: int64
2494


### 데이터프레임의 첫번째 열에 대해 개행문자를 삭제하는 코드

In [30]:
for i in range(len(grouped_df)):
    for j in range(len(grouped_df.iloc[i, 1])):
        grouped_df.iloc[i, 1][j] = grouped_df.iloc[i, 1][j].strip()

### 위원장 발언 삭제하는 코드(이하 2줄)

- 바로 아래 코드 : "위원장"이라는 단어가 처음 두 단어에 포함되어있는 원소들을 'word_list' 변수에 할당 word_list를 꼭 확인해보고 안에 이상한게 있지 않은지 검사 후 예외를 설정해야함.
- 그 다음 코드 : 'word_list' 변수에 있는 모든 단어가 처음 두 단어에 포함되어 있으면 삭제

# 아래 코드 일단 사용하지 않음
--------------------------------------------------------------

from collections import Counter

### '위원장'이 포함된 원소들의 처음 두 단어를 가져오는 함수
def get_elements_with_word(cell):
    if isinstance(cell, list):  # cell이 리스트 형인지 확인
        return [word for s in cell for word in re.split(' |,|\n', s)[:2] if "위원장" in word]
    else:
        return []

### 단어 빈도를 저장할 Counter 객체 초기화
word_freq = Counter()

### 0번째 열 제외하고 적용
for i in range(1, len(grouped_df.columns)):
    # 각 셀에서 '위원장'이 포함된 원소들의 처음 두 단어를 가져옴
    words = grouped_df.iloc[:, i].map(get_elements_with_word)
    # 각 셀에서 가져온 단어들의 빈도를 계산
    for word in words:
        word_freq.update(word)

word_list = [(word, freq) for word, freq in word_freq.most_common()]

def remove_attendance_elements(cell):
    if isinstance(cell, list):  # cell이 리스트 형인지 확인
        return [s for s in cell if not any(word for word in re.split(' |,|\n', s)[:2] if word in word_list)]
    else:
        return None

### 0번째 열 제외하고 적용
grouped_df.iloc[:, 1:] = grouped_df.iloc[:, 1:].applymap(remove_attendance_elements)

------------------

### 데이터 프레임 안에 비어있는 셀 정보 확인용

In [31]:
empty_count = grouped_df.iloc[:, 1:].isna().sum()
print(f"설정한 구간에서 각 컬럼별로 비어있는 셀 정보\n\n{empty_count}\n")

empty_rows = grouped_df[grouped_df.iloc[:, 1].isna()]
print(f"설정한 구간에서 로우별로 비어있는 셀 정보\n\n{empty_rows}")

설정한 구간에서 각 컬럼별로 비어있는 셀 정보

1    0
dtype: int64

설정한 구간에서 로우별로 비어있는 셀 정보

Empty DataFrame
Columns: [0, 1]
Index: []


In [32]:
grouped_df.head()

,0,1
0,2002년월드컵등국제경기대회지원특별위원회,[위원장 신경식 의석을 정돈해 주시기 바랍니다. 성원이 되었으므로 제5차 2002년...
1,IMF환란원인규명과경제위기진상조사를위한국정조사특별위원회,[위원장 장재식 자리를 정돈해 주시기 바랍니다. 성원이 되었으므로 제19차 IMF환...
2,건설교통위원회,[위원장 백남치 연일 수고가 많으십니다. 지금부터 헌법 제61조 국회법 제127조...
3,경제구조개혁및실업대책특별위원회,[위원장 정영훈 성원이 되었으므로 제5차 경제구조개혁및실업대책특별위원회를 개의하겠습...
4,과학기술정보통신위원회,[위원장 박우병 좌석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 제198회 국회...


In [33]:
grouped_df.shape

(42, 2)

In [34]:
grouped_df.iloc[0,1]

['위원장 신경식 의석을 정돈해 주시기 바랍니다. 성원이 되었으므로 제5차 2002년월드컵등국제경기대회지원특별위원회를 개의하겠습니다. 먼저 보고사항이 있겠습니다. 입법조사관 나와서 보고해 주시기 바랍니다.',
 '위원장 신경식 의사일정에 들어가기 전에 위원장의 신상에 관한 발언을 하고자 합니다. 존경하는 위원님 여러분  그 동안 우리 위원회는 위원님 여러분의 적극적인 참여와 의정활동에 힘입어 2002년월드컵축구대회지원법을 제정하는 등 국제경기대회지원을 위해 많은 활동을 해왔습니다. 얼마 전에 무주․전주유니버시아드대회가 성공적으로 끝났습니다마는 이제 부산동아시아경기대회가 금년 5월로 다가와 있고 그 외에도 99년 강원도동계아시아경기대회 등 각종 경기대회 지원을 위해서는 아직도 우리 특별위원회가 해야 할 일이 많습니다. 이러한 중요한 시기에 본인이 일신상의 이유로 위원장직을 수행할 수 없게 되어 오늘 위원회 전체회의를 통하여 사의를 표하고자 합니다. 국회법 제50조제4항의 규정에 따르면 위원장 궐위시 다수당의 간사께서 위원장직무대리로 위원회 주재를 할 수 있도록 되어 있으므로 신한국당 김기수 간사 위원님께서 사회를 맡아 주시기 바라며 저는 자리를 뜨겠습니다. 그 동안 정말로 감사했습니다. 저희 위원회가 자주 모이지는 못했습니다마는 모두 참 좋은 분들  마음으로 서로 믿고 도와주시고 격려해주신 덕분에 별 과오 없이 지내왔고 또 아까도 말씀드렸습니다마는 지금 우리 특위에서 해야 할 중요한 일들이 많은 이때에 같이 일을 하지 못하게 되어서 섭섭한 마음을 금치 못합니다마는  제가 위원회를 떠나더라도 우리 특위에 관한 일이라면 뒤에서나마 제가 할 수 있는 모든 지원을 열심히 하겠습니다. 그 동안 도와주시고 격려해 주신 데 대해서 다시 한번 진심으로 감사의 말씀을 드립니다. 고맙습니다. (신경식 위원장  김기수 간사와 사회교대)',
 '위원장직무대리 김기수 신한국당에서 간사직을 맡고 있는 김기수 위원입니다. 조금 전에 위원장께서 말씀을 드린 바와 같이 위원장의 지명에 의해서

In [35]:
# 16대의 경우 '최용규의원'처럼 규칙에 어긋난 리스트의 원소들이 있어서 이를 '최용규 의원'으로 바꾸는 코드를 만들었습니다.

def process_text(text, names):
    first_word = text.split()[0] # 첫 번째 단어를 찾는다
    for name in names:
        if name in first_word and '위원' in first_word: 
            # 첫 번째 단어에 'names'의 원소와 '위원'이 포함되어 있는지 확인한다
            new_word = first_word.replace(name, f"{name} ") 
            # 첫 번째 단어에 원소가 포함되어 있고 '위원'이라는 문자열이 포함되어 있는 경우 사이에 공백을 추가한다
            text = text.replace(first_word, new_word)
    return text

grouped_df.iloc[:, 1] = grouped_df.iloc[:, 1].apply(lambda x: [process_text(text, names) for text in x])

### 데이터 프레임의 구간을 설정하고 행별로 합치는 코드 - csv 만들기 편하게 구조 변경

In [36]:
grouped_df2 = grouped_df.copy()

In [37]:
# 0번째 컬럼 제외하고 모든 셀을 행별로 합치기
grouped_df2['merged'] = grouped_df2.iloc[:, 1:].apply(lambda row: [item for sublist in row if sublist is not None for item in sublist], axis=1)

# 이후 csv 파일로 저장하기 편한 구조로 만들기
final_df = grouped_df2[[0, 'merged']].rename(columns={"0": "Index", "merged": "Content"})

In [38]:
final_df.iloc[0,1]

['위원장 신경식 의석을 정돈해 주시기 바랍니다. 성원이 되었으므로 제5차 2002년월드컵등국제경기대회지원특별위원회를 개의하겠습니다. 먼저 보고사항이 있겠습니다. 입법조사관 나와서 보고해 주시기 바랍니다.',
 '위원장 신경식 의사일정에 들어가기 전에 위원장의 신상에 관한 발언을 하고자 합니다. 존경하는 위원님 여러분  그 동안 우리 위원회는 위원님 여러분의 적극적인 참여와 의정활동에 힘입어 2002년월드컵축구대회지원법을 제정하는 등 국제경기대회지원을 위해 많은 활동을 해왔습니다. 얼마 전에 무주․전주유니버시아드대회가 성공적으로 끝났습니다마는 이제 부산동아시아경기대회가 금년 5월로 다가와 있고 그 외에도 99년 강원도동계아시아경기대회 등 각종 경기대회 지원을 위해서는 아직도 우리 특별위원회가 해야 할 일이 많습니다. 이러한 중요한 시기에 본인이 일신상의 이유로 위원장직을 수행할 수 없게 되어 오늘 위원회 전체회의를 통하여 사의를 표하고자 합니다. 국회법 제50조제4항의 규정에 따르면 위원장 궐위시 다수당의 간사께서 위원장직무대리로 위원회 주재를 할 수 있도록 되어 있으므로 신한국당 김기수 간사 위원님께서 사회를 맡아 주시기 바라며 저는 자리를 뜨겠습니다. 그 동안 정말로 감사했습니다. 저희 위원회가 자주 모이지는 못했습니다마는 모두 참 좋은 분들  마음으로 서로 믿고 도와주시고 격려해주신 덕분에 별 과오 없이 지내왔고 또 아까도 말씀드렸습니다마는 지금 우리 특위에서 해야 할 중요한 일들이 많은 이때에 같이 일을 하지 못하게 되어서 섭섭한 마음을 금치 못합니다마는  제가 위원회를 떠나더라도 우리 특위에 관한 일이라면 뒤에서나마 제가 할 수 있는 모든 지원을 열심히 하겠습니다. 그 동안 도와주시고 격려해 주신 데 대해서 다시 한번 진심으로 감사의 말씀을 드립니다. 고맙습니다. (신경식 위원장  김기수 간사와 사회교대)',
 '위원장직무대리 김기수 신한국당에서 간사직을 맡고 있는 김기수 위원입니다. 조금 전에 위원장께서 말씀을 드린 바와 같이 위원장의 지명에 의해서

In [39]:
# final_df를 csv로 저장
final_df.to_csv('final.csv', index=False)

### 17대 국회의원 이름

- names_R_17 = 17대 보수당 국회의원
- names_L_17 = 17대 민주당 국회의원

In [40]:
names_L_15 = ["국창근",	"권기술",	"권노갑",	"권오을",	"길승흠",	"김경재",	"김근태",	"김명규",	"김민석",	"김병태",	"김봉호",	"김상우",	"김상현",	"김성곤",	"김영배",	"김영진",	"김영환",	"김옥두",	"김원길",	"김인곤",	"김종배",	"김진배",	"김충조",	"김태랑",	"김태식",	"김한길",	"김홍신",	"김홍일",	"남궁진",	"노무현",	"박광태",	"박상규",	"박상천",	"박정수",	"박정훈",	"박찬주",	"방용석",	"배종무",	"설훈",	"손세일",	"송현섭",	"신기남",	"신기하",	"신낙균",	"안동선",	"양성철",	"유선호",	"유재건",	"윤철상",	"이규정",	"이규택",	"이기문",	"이길재",	"이동원",	"이미경",	"이부영",	"이상수",	"이석현",	"이성재",	"이수인",	"이영일",	"이윤수",	"이중재",	"이해찬",	"이협",	"이훈평",	"임복진",	"임채정",	"장성원",	"장영달",	"장을병",	"장재식",	"정균환",	"정동영",	"정동채",	"정세균",	"정한용",	"정호선",	"정희경",	"제정구",	"조성준",	"조세형",	"조순승",	"조순형",	"조중연",	"조찬형",	"조철구",	"조한천",	"조홍규",	"채영석",	"천용택",	"천정배",	"최선영",	"최욱철",	"최재승",	"최희준",	"추미애",	"하경근",	"한광옥",	"한영애",	"한화갑",	"황규선"]
names_R_15 = ["강경식",	"강삼재",	"강성재",	"강용식",	"강재섭",	"강종희",	"강창희",	"강현욱",	"구천서",	"권수창",	"권영자",	"권익현",	"권철현",	"김고성",	"김광수",	"김광원",	"김기수",	"김기재",	"김기춘",	"김길환",	"김덕",	"김덕룡",	"김도언",	"김동욱",	"김동주",	"김명섭",	"김명윤",	"김무성",	"김문수",	"김범명",	"김복동",	"김석원",	"김선길",	"김수한",	"김영구",	"김영선",	"김영일",	"김영진",	"김용환",	"김운환",	"김윤환",	"김의재",	"김인영",	"김일주",	"김정수",	"김정숙",	"김종필",	"김종하",	"김종학",	"김종호",	"김중위",	"김진재",	"김찬우",	"김찬진",	"김철",	"김충일",	"김칠환",	"김태호",	"김학원",	"김허남",	"김현욱",	"김형오",	"김호일",	"김화남",	"나오연",	"남경필",	"남평우",	"노기태",	"노승우",	"류종수",	"맹형규",	"목요상",	"박관용",	"박구일",	"박근혜",	"박명환",	"박범진",	"박상복",	"박성범",	"박세직",	"박세환",	"박승국",	"박신원",	"박우병",	"박원홍",	"박종근",	"박종웅",	"박주천",	"박준규",	"박창달",	"박철언",	"박헌기",	"박희태",	"백남치",	"변웅전",	"변정일",	"서상목",	"서석재",	"서정화",	"서정화",	"서청원",	"서한샘",	"손학규",	"송업교",	"송훈석",	"신경식",	"신상우",	"신영국",	"신영균",	"심정구",	"안상수",	"안상수",	"안재홍",	"안택수",	"어준선",	"오세응",	"오양순",	"오용운",	"오장섭",	"유용태",	"유흥수",	"윤원중",	"윤한도",	"이강두",	"이강희",	"이건개",	"이경재",	"이국헌",	"이긍규",	"이동복",	"이만섭",	"이명박",	"이병희",	"이사철",	"이상득",	"이상만",	"이상배",	"이상현",	"이상희",	"이성호",	"이세기",	"이신범",	"이신행",	"이양희",	"이완구",	"이용삼",	"이우재",	"이웅희",	"이원범",	"이원복",	"이윤성",	"이응선",	"이의익",	"이인구",	"이재명",	"이재선",	"이재오",	"이재창",	"이정무",	"이찬진",	"이태섭",	"이택석",	"이한동",	"이해구",	"이형배",	"이홍구",	"이회창",	"임인배",	"장영철",	"전석홍",	"전용원",	"정문화",	"정상구",	"정상천",	"정석모",	"정영훈",	"정우택",	"정의화",	"정일영",	"정재문",	"정재철",	"정창화",	"정형근",	"조순",	"조영재",	"조웅규",	"조익현",	"조종석",	"조진형",	"주양자",	"주진우",	"지대섭",	"차수명",	"최병렬",	"최연희",	"최형우",	"하순봉",	"한승수",	"한영수",	"한이헌",	"한호선",	"함석재",	"함종한",	"허남훈",	"허대범",	"현경대",	"홍문종",	"홍인길",	"홍준표",	"황낙주",	"황병태",	"황승민",	"황우여",	"황학수",]

In [ ]:
names_L_16 = ["강봉균",	"강성구",	"강현욱",	"고진부",	"곽치영",	"구종태",	"김경재",	"김경천",	"김근태",	"김기재",	"김덕규",	"김덕배",	"김명섭",	"김민석",	"김방림",	"김상현",	"김성순",	"김성호",	"김영배",	"김영진",	"김영환",	"김옥두",	"김운용",	"김원기",	"김원길",	"김윤식",	"김충조",	"김태식",	"김태홍",	"김택기",	"김한길",	"김홍일",	"김화중",	"김효석",	"김희선",	"남궁석",	"문석호",	"문희상",	"박광태",	"박금자",	"박병석",	"박병윤",	"박상규",	"박상천",	"박상희",	"박양수",	"박용호",	"박인상",	"박종완",	"박종우",	"배기선",	"배기운",	"서영훈",	"설송웅",	"설훈",	"송석찬",	"송영길",	"송영진",	"송훈석",	"신계륜",	"신기남",	"심규섭",	"심재권",	"안동선",	"안상현",	"안희옥",	"양승부",	"오영식",	"원유철",	"유삼남",	"유용태",	"유재건",	"유재규",	"윤철상",	"이근진",	"이낙연",	"이만섭",	"이미경",	"이상수",	"이용삼",	"이원성",	"이윤수",	"이인제",	"이재정",	"이종걸",	"이종성",	"이창복",	"이해찬",	"이협",	"이호웅",	"이훈평",	"이희규",	"임종석",	"임채정",	"장성민",	"장성원",	"장영달",	"장영신",	"장재식",	"장정언",	"장태완",	"전갑길",	"전용학",	"정균환",	"정대철",	"정동영",	"정동채",	"정범구",	"정세균",	"정장선",	"정철기",	"조배숙",	"조성준",	"조순형",	"조재환",	"조한천",	"천용택",	"천정배",	"최명헌",	"최선영",	"최영희",	"최용규",	"최재승",	"추미애",	"한명숙",	"한충수",	"한화갑",	"함승희",	"허운나",	"홍재형",	"황창주"]
names_R_16 = ["강삼재",	"강신성일",	"강인섭",	"강재섭",	"강창성",	"강창희",	"고흥길",	"권기술",	"권영세",	"권오을",	"권철현",	"권태망",	"김광원",	"김기배",	"김기춘",	"김덕룡",	"김동욱",	"김락기",	"김만제",	"김무성",	"김문수",	"김병호",	"김부겸",	"김성조",	"김영구",	"김영구",	"김영선",	"김영일",	"김영춘",	"김용갑",	"김용균",	"김용학",	"김원웅",	"김일윤",	"김정부",	"김정숙",	"김종필",	"김종하",	"김종호",	"김진재",	"김찬우",	"김태호",	"김학송",	"김학원",	"김형오",	"김호일",	"김홍신",	"김황식",	"나오연",	"남경필",	"도종이",	"맹형규",	"목요상",	"민봉기",	"박경섭",	"박관용",	"박근혜",	"박명환",	"박세환",	"박승국",	"박시균",	"박원홍",	"박재욱",	"박종근",	"박종웅",	"박종희",	"박주천",	"박진",	"박창달",	"박헌기",	"박혁규",	"박희태",	"백승홍",	"변웅전",	"서병수",	"서상섭",	"서정화",	"서청원",	"손태인",	"손학규",	"손희정",	"송광호",	"송병대",	"신경식",	"신영국",	"신영균",	"신현태",	"심규철",	"심재철",	"안경률",	"안대륜",	"안상수",	"안영근",	"안택수",	"양정규",	"엄호성",	"오경훈",	"오세훈",	"오장섭",	"원철희",	"원희룡",	"유성근",	"유한열",	"유흥수",	"윤경식",	"윤두환",	"윤여준",	"윤영탁",	"윤한도",	"이강두",	"이경재",	"이규택",	"이만재",	"이방호",	"이병석",	"이부영",	"이상득",	"이상배",	"이상희",	"이성헌",	"이승철",	"이양희",	"이연숙",	"이완구",	"이우재",	"이원창",	"이원형",	"이윤성",	"이인기",	"이재선",	"이재오",	"이재창",	"이주영",	"이한구",	"이한동",	"이해구",	"이해봉",	"이회창",	"임인배",	"임진출",	"임태희",	"장광근",	"전용원",	"전재희",	"정갑윤",	"정문화",	"정병국",	"정우택",	"정의화",	"정인봉",	"정재문",	"정진석",	"정창화",	"정형근",	"조부영",	"조웅규",	"조정무",	"조희욱",	"주진우",	"최돈웅",	"최병국",	"최병렬",	"최연희",	"하순봉",	"함석재",	"허태열",	"현경대",	"현승일",	"홍문종",	"홍사덕",	"홍준표",	"황승민",	"황우여"]

In [ ]:
names_L_17 = ["강기정",	"강길부",	"강봉균",	"강성종",	"강창일",	"강혜숙",	"구논회",	"권선택",	"김교흥",	"김근태",	"김기석",	"김낙순",	"김덕규",	"김동철",	"김맹곤",	"김명자",	"김부겸",	"김선미",	"김성곤",	"김송자",	"김영주",	"김영춘",	"김우남",	"김원기",	"김원웅",	"김재윤",	"김재홍",	"김종률",	"김진표",	"김춘진",	"김태년",	"김태홍",	"김한길",	"김혁규",	"김현미",	"김형주",	"김홍업",	"김희선",	"노영민",	"노웅래",	"노현송",	"문병호",	"문석호",	"문학진",	"문희상",	"민병두",	"박기춘",	"박명광",	"박병석",	"박상돈",	"박영선",	"박찬석",	"박홍수",	"배기선",	"백원우",	"변재일",	"복기왕",	"서갑원",	"서재관",	"서혜석",	"선병렬",	"송영길",	"신계륜",	"신기남",	"신명",	"신중식",	"신학용",	"심재덕",	"안민석",	"안병엽",	"안영근",	"양승조",	"양형일",	"염동연",	"오시덕",	"오영식",	"오제세",	"우상호",	"우원식",	"우윤근",	"우제창",	"우제항",	"원혜영",	"유기홍",	"유선호",	"유승희",	"유시민",	"유인태",	"유재건",	"유필우",	"윤원호",	"윤호중",	"이강래",	"이경숙",	"이계안",	"이광재",	"이광철",	"이근식",	"이기우",	"이목희",	"이미경",	"이상경",	"이상락",	"이상민",	"이석현",	"이시종",	"이영호",	"이용희",	"이원영",	"이은영",	"이인영",	"이종걸",	"이철우",	"이해찬",	"이호웅",	"이화영",	"임종석",	"임종인",	"임채정",	"장경수",	"장복심",	"장영달",	"장향숙",	"전병헌",	"정덕구",	"정동채",	"정봉주",	"정성호",	"정세균",	"정의용",	"정장선",	"정청래",	"제종길",	"조경태",	"조배숙",	"조성래",	"조성태",	"조순형",	"조일현",	"조정식",	"주승용",	"지병문",	"채수찬",	"채일병",	"천정배",	"최규성",	"최규식",	"최성",	"최용규",	"최재성",	"최재천",	"최철국",	"한광원",	"한명숙",	"한병도",	"홍미영",	"홍재형",	"홍창선"]
names_R_17 = ["강재섭", 	"고경화", 	"고조흥", 	"고진화", 	"고흥길", 	"고희선", 	"공성진", 	"곽성문", 	"권경석", 	"권영세", 	"권오을", 	"권철현", 	"김광원", 	"김기춘", 	"김기현", 	"김낙성", 	"김덕룡", 	"김명주", 	"김무성", 	"김문수", 	"김병호", 	"김석준", 	"김성조", 	"김애실", 	"김양수", 	"김영덕", 	"김영선", 	"김영숙", 	"김용갑", 	"김재경", 	"김재원", 	"김정권", 	"김정부", 	"김정훈", 	"김충환", 	"김태환", 	"김학송", 	"김학원", 	"김형오", 	"김희정", 	"나경원", 	"남경필", 	"류근찬", 	"맹형규", 	"문희", 	"박계동", 	"박근혜", 	"박성범", 	"박세일", 	"박세환", 	"박순자", 	"박승환", 	"박재완", 	"박종근", 	"박진", 	"박찬숙", 	"박창달", 	"박혁규", 	"박형준", 	"박희태", 	"배일도", 	"서병수", 	"서상기", 	"송영선", 	"신상진", 	"심재엽", 	"심재철", 	"안경률", 	"안명옥", 	"안상수", 	"안택수", 	"안홍준", 	"엄호성", 	"원희룡", 	"유기준", 	"유승민", 	"유정복", 	"윤건영", 	"윤두환", 	"이강두", 	"이경재", 	"이계경", 	"이계진", 	"이군현", 	"이규택", 	"이덕모", 	"이명규", 	"이방호", 	"이병석", 	"이상득", 	"이상배", 	"이성구", 	"이성권", 	"이원복", 	"이윤성", 	"이인기", 	"이인제", 	"이재오", 	"이재웅", 	"이재창", 	"이종구", 	"이주영", 	"이주호", 	"이진구", 	"이한구", 	"이해봉", 	"이혜훈", 	"임인배", 	"임태희", 	"임해규", 	"장윤석", 	"전여옥", 	"전재희", 	"정갑윤", 	"정두언", 	"정문헌", 	"정병국", 	"정의화", 	"정종복", 	"정진섭", 	"정형근", 	"정화원", 	"정희수", 	"주성영", 	"주호영", 	"진수희", 	"진영", 	"차명진", 	"최경환", 	"최구식", 	"최병국", 	"최연희", 	"한선교", 	"허천", 	"허태열", 	"홍문표", 	"홍준표", 	"황우여", 	"황진하" ]

In [ ]:
names_L_18 = ["강기정",	"강봉균",	"강성종",	"강창일",	"김동철",	"김부겸",	"김상희",	"김성곤",	"김성순",	"김세웅",	"김영진",	"김영환",	"김우남",	"김유정",	"김재균",	"김재윤",	"김종률",	"김진애",	"김진표",	"김춘진",	"김충조",	"김학재",	"김효석",	"김희철",	"노영민",	"문학진",	"문희상",	"박기춘",	"박병석",	"박상천",	"박선숙",	"박영선",	"박우순",	"박은수",	"박주선",	"백원우",	"백재현",	"변재일",	"서갑원",	"서종표",	"손학규",	"송민순",	"송영길",	"신낙균",	"신학용",	"안규백",	"안민석",	"양승조",	"오제세",	"우윤근",	"우제창",	"원혜영",	"유선호",	"이강래",	"이광재",	"이낙연",	"이미경",	"이석현",	"이성남",	"이시종",	"이용삼",	"이용섭",	"이종걸",	"이찬열",	"이춘석",	"장병완",	"장세환",	"전병헌",	"전현희",	"전혜숙",	"정국교",	"정범구",	"정세균",	"정장선",	"조경태",	"조배숙",	"조영택",	"조정식",	"주승용",	"천정배",	"최규성",	"최규식",	"최문순",	"최영희",	"최인기",	"최재성",	"최종원",	"최철국",	"추미애",	"홍영표",	"홍재형"]
names_R_18 = ["강명순",	"강석호",	"강성천",	"강승규",	"강용석",	"고승덕",	"고흥길",	"공성진",	"구본철",	"구상찬",	"권경석",	"권선택",	"권성동",	"권영세",	"권영진",	"권택기",	"김금래",	"김기현",	"김낙성",	"김동성",	"김선동",	"김성동",	"김성수",	"김성식",	"김성조",	"김성태",	"김성회",	"김소남",	"김영선",	"김영우",	"김옥이",	"김용구",	"김용태",	"김장수",	"김재경",	"김정권",	"김정훈",	"김창수",	"김충환",	"김태원",	"김태호",	"김학송",	"김학용",	"김형오",	"김호연",	"김효재",	"나경원",	"나성린",	"남경필",	"류근찬",	"박근혜",	"박민식",	"박보환",	"박상돈",	"박상은",	"박선영",	"박순자",	"박영아",	"박종희",	"박준선",	"박진",	"박희태",	"배영식",	"배은희",	"백성운",	"변웅전",	"서병수",	"서상기",	"손범규",	"손숙미",	"송광호",	"신상진",	"신성범",	"신영수",	"신지호",	"심대평",	"심재철",	"안경률",	"안상수",	"안형환",	"안홍준",	"안효대",	"여상규",	"원유철",	"원희룡",	"원희목",	"유승민",	"유일호",	"유정복",	"유정현",	"윤두환",	"윤상현",	"윤석용",	"윤영",	"윤진식",	"이계진",	"이군현",	"이달곤",	"이두아",	"이명규",	"이명수",	"이범관",	"이범래",	"이병석",	"이사철",	"이상권",	"이상득",	"이상민",	"이성헌",	"이애주",	"이영애",	"이영애",	"이용희",	"이윤성",	"이은재",	"이재선",	"이재오",	"이정선",	"이정현",	"이종구",	"이종혁",	"이주영",	"이진삼",	"이철우",	"이춘식",	"이학재",	"이한구",	"이한성",	"이혜훈",	"이화수",	"이회창",	"임동규",	"임두성",	"임영호",	"임태희",	"임해규",	"장광근",	"장윤석",	"장제원",	"전여옥",	"전재희",	"정갑윤",	"정두언",	"정몽준",	"정미경",	"정병국",	"정양석",	"정옥임",	"정의화",	"정진석",	"정진섭",	"정태근",	"정희수",	"조문환",	"조순형",	"조윤선",	"조전혁",	"조진래",	"조진형",	"조해진",	"주광덕",	"주성영",	"주호영",	"진성호",	"진수희",	"진영",	"차명진",	"최경환",	"최경희",	"최병국",	"한기호",	"허범도",	"허원제",	"허천",	"허태열",	"현경병",	"현기환",	"홍일표",	"홍정욱",	"홍준표",	"황영철",	"황우여",	"황진하"]

In [ ]:
names_L_19 = ["강기정",	"강창일",	"권은희",	"김경협",	"김관영",	"김광진",	"김기식",	"김기준",	"김동철",	"김민기",	"김상희",	"김성곤",	"김성주",	"김승남",	"김영록",	"김영주",	"김영환",	"김용익",	"김우남",	"김윤덕",	"김재윤",	"김진표",	"김춘진",	"김태년",	"김한길",	"김현",	"김현미",	"남인순",	"노영민",	"노웅래",	"도종환",	"문병호",	"문재인",	"문희상",	"민병두",	"민홍철",	"박광온",	"박기춘",	"박남춘",	"박민수",	"박범계",	"박병석",	"박수현",	"박영선",	"박완주",	"박지원",	"박혜자",	"박홍근",	"배기운",	"배재정",	"백군기",	"백재현",	"변재일",	"부좌현",	"서영교",	"설훈",	"송호창",	"신경민",	"신계륜",	"신기남",	"신문식",	"신장용",	"신정훈",	"신학용",	"심재권",	"안규백",	"안민석",	"양승조",	"오영식",	"오제세",	"우상호",	"우원식",	"우윤근",	"원혜영",	"유기홍",	"유대운",	"유승희",	"유은혜",	"유인태",	"윤관석",	"윤호중",	"윤후덕",	"은수미",	"이개호",	"이낙연",	"이목희",	"이미경",	"이상민",	"이상직",	"이석현",	"이언주",	"이용섭",	"이원욱",	"이윤석",	"이인영",	"이종걸",	"이찬열",	"이춘석",	"이학영",	"이해찬",	"인재근",	"임내현",	"임수경",	"장병완",	"장하나",	"전병헌",	"전순옥",	"전정희",	"전해철",	"정성호",	"정세균",	"정청래",	"정호준",	"조경태",	"조정식",	"주승용",	"진선미",	"진성준",	"최규성",	"최동익",	"최민희",	"최원식",	"최재성",	"최재천",	"추미애",	"한명숙",	"한정애",	"홍영표",	"홍의락",	"홍익표",	"홍종학",	"황주홍"]
names_R_19 = ["강기윤",	"강길부",	"강석호",	"강석훈",	"강은희",	"강창희",	"경대수",	"고희선",	"권성동",	"권은희",	"길정우",	"김광림",	"김근태",	"김기선",	"김기현",	"김도읍",	"김동완",	"김명연",	"김무성",	"김상민",	"김상훈",	"김성찬",	"김성태",	"김세연",	"김영우",	"김영주",	"김용남",	"김용태",	"김을동",	"김장실",	"김재경",	"김재원",	"김정록",	"김정훈",	"김제식",	"김종태",	"김종훈",	"김진태",	"김태원",	"김태호",	"김태환",	"김태흠",	"김학용",	"김현숙",	"김형태",	"김회선",	"김희국",	"김희정",	"나경원",	"나성린",	"남경필",	"노철래",	"류성걸",	"류지영",	"문대성",	"문정림",	"민병주",	"민현주",	"박근혜",	"박대동",	"박대출",	"박덕흠",	"박맹우",	"박명재",	"박민식",	"박상은",	"박성호",	"박성효",	"박윤옥",	"박인숙",	"박창식",	"배덕광",	"서병수",	"서상기",	"서용교",	"서청원",	"성완종",	"손인춘",	"송광호",	"송영근",	"신경림",	"신동우",	"신상진",	"신성범",	"신의진",	"심윤조",	"심재철",	"심학봉",	"안덕수",	"안상수",	"안종범",	"안홍준",	"안효대",	"양창영",	"여상규",	"염동열",	"오신환",	"원유철",	"유기준",	"유승민",	"유승우",	"유의동",	"유일호",	"유재중",	"유정복",	"윤명희",	"윤상현",	"윤영석",	"윤재옥",	"윤진식",	"이강후",	"이군현",	"이노근",	"이만우",	"이명수",	"이병석",	"이상일",	"이에리사",	"이완구",	"이완영",	"이우현",	"이운룡",	"이이재",	"이인제",	"이자스민",	"이장우",	"이재균",	"이재영",	"이재영",	"이재오",	"이정현",	"이종배",	"이종진",	"이종훈",	"이주영",	"이진복",	"이채익",	"이철우",	"이학재",	"이한구",	"이한성",	"이헌승",	"이현재",	"장윤석",	"장정은",	"전하진",	"정갑윤",	"정두언",	"정몽준",	"정문헌",	"정미경",	"정병국",	"정수성",	"정용기",	"정우택",	"정윤숙",	"정의화",	"정희수",	"조명철",	"조원진",	"조해진",	"조현룡",	"주영순",	"주호영",	"진영",	"최경환",	"최봉홍",	"하태경",	"한기호",	"한선교",	"함진규",	"현영희",	"홍문종",	"홍문표",	"홍일표",	"홍지만",	"홍철호",	"황영철",	"황우여",	"황인자",	"황진하",]

In [ ]:
names_L_20 = ["강석진",	"강석호",	"강효상",	"경대수",	"곽대훈",	"곽상도",	"권석창",	"권성동",	"김광림",	"김규환",	"김기선",	"김도읍",	"김명연",	"김무성",	"김상훈",	"김석기",	"김선동",	"김성원",	"김성찬",	"김성태",	"김성태",	"김세연",	"김순례",	"김승희",	"김영우",	"김용태",	"김재경",	"김재원",	"김정재",	"김정훈",	"김종석",	"김종태",	"김진태",	"김태흠",	"김학용",	"김한표",	"김현아",	"나경원",	"문진국",	"민경욱",	"박대출",	"박덕흠",	"박맹우",	"박명재",	"박성중",	"박순자",	"박완수",	"박인숙",	"박찬우",	"배덕광",	"백승주",	"서청원",	"성일종",	"송석준",	"송언석",	"송희경",	"신보라",	"신상진",	"심재철",	"엄용수",	"여상규",	"염동열",	"오신환",	"원유철",	"유기준",	"유민봉",	"유의동",	"유재중",	"윤상직",	"윤영석",	"윤재옥",	"윤종필",	"윤한홍",	"이군현",	"이만희",	"이명수",	"이양수",	"이완영",	"이우현",	"이은권",	"이은재",	"이장우",	"이정현",	"이종구",	"이종명",	"이종배",	"이주영",	"이진복",	"이채익",	"이철우",	"이학재",	"이헌승",	"이현재",	"이혜훈",	"임이자",	"임재훈",	"장석춘",	"전희경",	"정갑윤",	"정병국",	"정양석",	"정용기",	"정우택",	"정운천",	"정유섭",	"정점식",	"정종섭",	"정진석",	"정태옥",	"조경태",	"조원진",	"조훈현",	"주광덕",	"지상욱",	"최경환",	"최교일",	"최연혜",	"추경호",	"하태경",	"한선교",	"함진규",	"홍문종",	"홍문표",	"홍일표",	"홍철호",	"황영철",	"홍일표",	"홍지만",	"홍철호",	"황영철",	"황우여",	"황인자",	"황진하"]
names_R_20 = ["강병원",	"강창일",	"강훈식",	"고용진",	"권미혁",	"권칠승",	"금태섭",	"기동민",	"김경수",	"김경협",	"김두관",	"김민기",	"김병관",	"김병기",	"김병욱",	"김부겸",	"김상희",	"김성수",	"김성환",	"김영주",	"김영진",	"김영춘",	"김영호",	"김정우",	"김정호",	"김종민",	"김종인",	"김진표",	"김철민",	"김태년",	"김한정",	"김해영",	"김현권",	"김현미",	"남인순",	"노웅래",	"도종환",	"맹성규",	"문미옥",	"문희상",	"민병두",	"민홍철",	"박경미",	"박광온",	"박남춘",	"박범계",	"박병석",	"박영선",	"박완주",	"박용진",	"박재호",	"박정",	"박주민",	"박찬대",	"박홍근",	"백재현",	"백혜련",	"변재일",	"서삼석",	"서영교",	"서형수",	"설훈",	"소병훈",	"손혜원",	"송갑석",	"송기헌",	"송영길",	"송옥주",	"신경민",	"신동근",	"신창현",	"심기준",	"심재권",	"안규백",	"안민석",	"안호영",	"양승조",	"어기구",	"오영훈",	"오제세",	"우상호",	"우원식",	"원혜영",	"위성곤",	"유동수",	"유승희",	"유은혜",	"윤관석",	"윤일규",	"윤준호",	"윤호중",	"윤후덕",	"이개호",	"이규희",	"이상민",	"이상헌",	"이석현",	"이수혁",	"이언주",	"이용득",	"이원욱",	"이인영",	"이재정",	"이종걸",	"이찬열",	"이철희",	"이춘석",	"이학영",	"이후삼",	"이훈",	"인재근",	"임종성",	"전재수",	"전해철",	"전현희",	"전혜숙",	"정성호",	"정세균",	"정은혜",	"정재호",	"정춘숙",	"제윤경",	"조승래",	"조응천",	"조정식",	"진선미",	"진영",	"최명길",	"최운열",	"최인호",	"최재성",	"추미애",	"표창원",	"한정애",	"허윤정",	"홍영표",	"홍익표",	"황희",	"홍영표",	"홍의락",	"홍익표",	"홍종학",	"황주홍"]

In [41]:
def check_and_replace(element):
    words = element.split()
    first_two_words = ' '.join(words[:2])
    
    if any(name in first_two_words for name in names_R_15):
        return '1 ' + ' '.join(words[2:])
    elif any(name in first_two_words for name in names_L_15):
        return '0 ' + ' '.join(words[2:])
    else:
        return element  # If no match is found, return the original element

final_df['Content'] = final_df['Content'].apply(lambda row: [check_and_replace(item) for item in row])

In [42]:
final_df

,0,Content
0,2002년월드컵등국제경기대회지원특별위원회,[1 의석을 정돈해 주시기 바랍니다. 성원이 되었으므로 제5차 2002년월드컵등국제...
1,IMF환란원인규명과경제위기진상조사를위한국정조사특별위원회,[0 자리를 정돈해 주시기 바랍니다. 성원이 되었으므로 제19차 IMF환란조사특별위...
2,건설교통위원회,[1 연일 수고가 많으십니다. 지금부터 헌법 제61조 국회법 제127조 국정감사및조...
3,경제구조개혁및실업대책특별위원회,[1 성원이 되었으므로 제5차 경제구조개혁및실업대책특별위원회를 개의하겠습니다. 위원...
4,과학기술정보통신위원회,[1 좌석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 제198회 국회 정기회 제...
5,교육위원회,[1 그러면 의석을 정돈해 주시기 바랍니다. 성원이 되었으므로 제2차 교육위원회를 ...
6,국방위원회,[1 지금부터 헌법 제61조 국회법 제127조와 국정감사및조사에관한법률에 의하여 해...
7,국제경기대회지원특별위원회,[1 의석을 정돈해 주시기 바랍니다. 성원이 되었으므로 제2차 국제경기대회지원특별위...
8,국제경쟁력강화및경제제도개혁에관한특별위원회,[0 15대 국회에 들어오니까 내가 세 번째 연장자라고 해서 다른 사람이 놀라기 전...
9,국회본회의,[1 신한국당 소속 예산결산특별위원회 이강두 위원입니다. 존경하는 의장 그리고 선배...


In [43]:
# Step 1: Concatenate all items in the 'Content' lists into a new list 'the_contents'
the_contents = []
for lst in final_df['Content']:
    the_contents.extend(lst)

# Step 2: The 'the_contents' list now contains all elements of the 'Content' column in 'final_df'
# print(the_contents) to check

# Step 3: Create a DataFrame from 'the_contents'
df_contents = pd.DataFrame(the_contents, columns=['Content'])

In [44]:
the_contents

['1 의석을 정돈해 주시기 바랍니다. 성원이 되었으므로 제5차 2002년월드컵등국제경기대회지원특별위원회를 개의하겠습니다. 먼저 보고사항이 있겠습니다. 입법조사관 나와서 보고해 주시기 바랍니다.',
 '1 의사일정에 들어가기 전에 위원장의 신상에 관한 발언을 하고자 합니다. 존경하는 위원님 여러분 그 동안 우리 위원회는 위원님 여러분의 적극적인 참여와 의정활동에 힘입어 2002년월드컵축구대회지원법을 제정하는 등 국제경기대회지원을 위해 많은 활동을 해왔습니다. 얼마 전에 무주․전주유니버시아드대회가 성공적으로 끝났습니다마는 이제 부산동아시아경기대회가 금년 5월로 다가와 있고 그 외에도 99년 강원도동계아시아경기대회 등 각종 경기대회 지원을 위해서는 아직도 우리 특별위원회가 해야 할 일이 많습니다. 이러한 중요한 시기에 본인이 일신상의 이유로 위원장직을 수행할 수 없게 되어 오늘 위원회 전체회의를 통하여 사의를 표하고자 합니다. 국회법 제50조제4항의 규정에 따르면 위원장 궐위시 다수당의 간사께서 위원장직무대리로 위원회 주재를 할 수 있도록 되어 있으므로 신한국당 김기수 간사 위원님께서 사회를 맡아 주시기 바라며 저는 자리를 뜨겠습니다. 그 동안 정말로 감사했습니다. 저희 위원회가 자주 모이지는 못했습니다마는 모두 참 좋은 분들 마음으로 서로 믿고 도와주시고 격려해주신 덕분에 별 과오 없이 지내왔고 또 아까도 말씀드렸습니다마는 지금 우리 특위에서 해야 할 중요한 일들이 많은 이때에 같이 일을 하지 못하게 되어서 섭섭한 마음을 금치 못합니다마는 제가 위원회를 떠나더라도 우리 특위에 관한 일이라면 뒤에서나마 제가 할 수 있는 모든 지원을 열심히 하겠습니다. 그 동안 도와주시고 격려해 주신 데 대해서 다시 한번 진심으로 감사의 말씀을 드립니다. 고맙습니다. (신경식 위원장 김기수 간사와 사회교대)',
 '1 신한국당에서 간사직을 맡고 있는 김기수 위원입니다. 조금 전에 위원장께서 말씀을 드린 바와 같이 위원장의 지명에 의해서 제가 사회를 맡게 되었습니다. 조금 전에 신상

In [45]:
df_contents

,Content
0,1 의석을 정돈해 주시기 바랍니다. 성원이 되었으므로 제5차 2002년월드컵등국제경...
1,1 의사일정에 들어가기 전에 위원장의 신상에 관한 발언을 하고자 합니다. 존경하는 ...
2,1 신한국당에서 간사직을 맡고 있는 김기수 위원입니다. 조금 전에 위원장께서 말씀을...
3,1 그러면 의사일정 제1항 위원장(신경식) 사임의 건을 상정하겠습니다. 신경식 위원...
4,1 다음은 의사일정 제2항 위원장 선임의 건을 상정하겠습니다. 위원 여러분께서 잘 ...
...,...
417398,1 질의 없으시지요? 그러면 내일 오전 11시에 상임위원회를 법안통과때문에 또 열어...
417399,1 하 요술방망이를 가진 회사네요.
417400,1 그러니까 의혹이 있는 거지요 사법부에다가 분명히 이야기를 해요.
417401,1 묘한 세상에 사시느라고 환경부장관 이하 국장님들 고생 많으신데 이 문제는 명분이...


In [46]:
# Split the 'Content' column based on the first space and expand into new dataframe
split_df = df_contents['Content'].str.split(' ', n=1, expand=True)

# Assign the first split to 'L|R' column and second split to 'Content' column
df_contents['L|R'] = split_df[0]
df_contents['Content'] = split_df[1]

In [47]:
df_contents

,Content,L|R
0,의석을 정돈해 주시기 바랍니다. 성원이 되었으므로 제5차 2002년월드컵등국제경기대...,1
1,의사일정에 들어가기 전에 위원장의 신상에 관한 발언을 하고자 합니다. 존경하는 위원...,1
2,신한국당에서 간사직을 맡고 있는 김기수 위원입니다. 조금 전에 위원장께서 말씀을 드...,1
3,그러면 의사일정 제1항 위원장(신경식) 사임의 건을 상정하겠습니다. 신경식 위원장님...,1
4,다음은 의사일정 제2항 위원장 선임의 건을 상정하겠습니다. 위원 여러분께서 잘 아시...,1
...,...,...
417398,질의 없으시지요? 그러면 내일 오전 11시에 상임위원회를 법안통과때문에 또 열어야 ...,1
417399,하 요술방망이를 가진 회사네요.,1
417400,그러니까 의혹이 있는 거지요 사법부에다가 분명히 이야기를 해요.,1
417401,묘한 세상에 사시느라고 환경부장관 이하 국장님들 고생 많으신데 이 문제는 명분이 있...,1


In [48]:
# Getting the values and their frequencies in 'L|R' column
print(df_contents['L|R'].value_counts())

L|R
1                      248572
0                      153523
김재천                      2168
위원장                      1837
백승홍                      1486
위원장대리                    1183
황성균                      1165
서훈                       1157
김용갑                      1086
임진출                      1066
박종우                       878
박시균                       507
이해봉                       456
김영준                       440
정몽준                       427
권정달                       313
홍사덕                       284
원유철                       260
양정규                       220
김일윤                       160
소위원장                       39
전문위원                       33
위원장직무대리                    33
행정자치위원장대리                  30
보건복지위원장대리                  27
수석전문위원                     21
서울지방노동위원장                   7
내무위원장대리                     6
박태준                         6
건설교통위원장대리                   2
재정경제위원장대리                   2
제14회부산아시아경기대회조직위원장          2
환경노동위원장대리                   2
문화체육공보

In [49]:
# Keeping only the rows where 'L|R' is '0' or '1'
df_contents = df_contents[df_contents['L|R'].isin(['0', '1'])]

In [50]:
df_contents = df_contents.reindex(columns=['L|R', 'Content'])

In [51]:
df_contents = df_contents.reset_index(drop=True)

In [52]:
df_contents

,L|R,Content
0,1,의석을 정돈해 주시기 바랍니다. 성원이 되었으므로 제5차 2002년월드컵등국제경기대...
1,1,의사일정에 들어가기 전에 위원장의 신상에 관한 발언을 하고자 합니다. 존경하는 위원...
2,1,신한국당에서 간사직을 맡고 있는 김기수 위원입니다. 조금 전에 위원장께서 말씀을 드...
3,1,그러면 의사일정 제1항 위원장(신경식) 사임의 건을 상정하겠습니다. 신경식 위원장님...
4,1,다음은 의사일정 제2항 위원장 선임의 건을 상정하겠습니다. 위원 여러분께서 잘 아시...
...,...,...
402090,1,질의 없으시지요? 그러면 내일 오전 11시에 상임위원회를 법안통과때문에 또 열어야 ...
402091,1,하 요술방망이를 가진 회사네요.
402092,1,그러니까 의혹이 있는 거지요 사법부에다가 분명히 이야기를 해요.
402093,1,묘한 세상에 사시느라고 환경부장관 이하 국장님들 고생 많으신데 이 문제는 명분이 있...


In [53]:
file_path = 'D:\MJ 데이터 공모전 로우 데이터\\결과파일\\15대 민주당_보수당 구분.csv'
df_contents.to_csv(file_path, index=False)

----
----
----
----

# 이번 데이터 전처리는 여기까지
----
----
----
----

### dt_1과 dt_0으로 나누어 content만 저장하는 코드

In [ ]:
dt_1 = [item for sublist in final_df['Content'] for item in sublist if item.startswith('1 ')]
dt_0 = [item for sublist in final_df['Content'] for item in sublist if item.startswith('2 ')]

In [ ]:
the_content = [[item for sublist in final_df['Content'] for item in sublist if item.startswith('1 '|'0 ')]]

In [ ]:
import sys

print("Memory size of dt_1 in bytes: ", sys.getsizeof(dt_1))
print("Memory size of dt_0 in bytes: ", sys.getsizeof(dt_0))

In [ ]:
print(len(dt_1))
print(len(dt_0))

In [ ]:
dt_1

In [ ]:
dt_0

In [ ]:
dt_1 = [s.replace('1 ', '', 1) for s in dt_1]
dt_0 = [s.replace('2 ', '', 1) for s in dt_0]

In [ ]:
import numpy as np

# 두 리스트의 길이 비교
len_dt_1 = len(dt_1)
len_dt_0 = len(dt_0)

# 더 짧은 리스트에 NaN 값 추가
if len_dt_1 < len_dt_0:
    dt_1 += [np.nan] * (len_dt_0 - len_dt_1)
elif len_dt_0 < len_dt_1:
    dt_0 += [np.nan] * (len_dt_1 - len_dt_0)

In [ ]:
# 데이터프레임 생성
L_R_17 = pd.DataFrame({
    '보수': dt_1,
    '진보': dt_0
})

# CSV 파일로 저장
L_R_17.to_csv('17대 보수, 진보 발언 총합.csv', index=False, encoding='utf-8-sig')

### 데이터 프레임 열과 행 바꾸는 코드

In [ ]:
# 데이터프레임을 전치 (Transpose)
transposed_df = grouped_df2.transpose()

# 새로운 열 이름 설정
new_header = transposed_df.iloc[0] 

# 첫 번째 행을 삭제하고 새로운 헤더를 설정
transposed_df = transposed_df[1:] 
transposed_df.columns = new_header

### 사람 이름 추출하는 코드

In [ ]:
# 문자열에서 처음 두 단어 추출
new_df['first_two_words'] = new_df['value'].str.split().str[0:2].str.join(' ')

# 처음 두 단어 삭제
new_df['value'] = new_df['value'].apply(lambda x: ' '.join(x.split(' ')[2:]))

### csv 파일 저장

In [ ]:
file_path = 'D:\MJ 데이터 공모전 로우 데이터\\deleted_cells_df.csv'
deleted_cells_df.to_csv(file_path, index=False)

---
---
# 국회의원 명단 전처리
---
---

In [ ]:
# CSV 파일 경로
file_path = 'D:\\MJ 데이터 공모전 로우 데이터\\국회의원 명단\\15대 이후 국회의원 총 명단.csv'

# CSV 파일 불러오기
C_members = pd.read_csv(file_path, encoding='utf-8', header=0)

In [ ]:
# 정규 표현식 패턴: 괄호 안의 모든 문자를 찾습니다.
pattern = "\((.*?)\)"

# '대별 및 소속정당(단체)' 컬럼에 대하여, 패턴에 맞는 문자열을 찾아서 새로운 컬럼으로 추가합니다.
C_members['지역구'] = C_members['대별 및 소속정당(단체)'].apply(lambda x: re.findall(pattern, x))

# '대별 및 소속정당(단체)' 컬럼에서 패턴에 맞는 문자열을 제거합니다.
C_members['대별 및 지역'] = C_members['대별 및 소속정당(단체)'].apply(lambda x: re.sub(pattern, '', x))

In [ ]:
C_members

In [ ]:
# 정규 표현식 패턴: 괄호 안의 모든 문자를 찾습니다.
pattern = "국회의원"

# '대별 및 소속정당(단체)' 컬럼에 대하여, 패턴에 맞는 문자열을 찾아서 새로운 컬럼으로 추가합니다.
C_members['지역구'] = C_members['대별 및 지역'].apply(lambda x: re.findall(pattern, x))

# '대별 및 소속정당(단체)' 컬럼에서 패턴에 맞는 문자열을 제거합니다.
C_members['대별 및 지역'] = C_members['대별 및 소속정당(단체)'].apply(lambda x: re.sub(pattern, '', x))

In [ ]:
C_members

In [ ]:
# '국회의원'이 몇 번 나오는지 확인하여 새로운 컬럼 생성
split_column = C_members['대별 및 지역1'].str.split(' ', expand=True)
num_occurrences = split_column.shape[1] - 1  # '국회의원'이 나오는 횟수

# '두 번째 컬럼'에 '국회의원' 앞의 단어 할당
C_members['대별 및 지역1'] = split_column[0]

# 새로운 컬럼 생성하여 '국회의원' 뒤의 단어 할당
for i in range(num_occurrences):
    column_name = f'대별 및 지역1-{i+1}'
    C_members[column_name] = split_column[i+1]

# 출력
print(C_members)

In [ ]:
C_members

In [ ]:
import pandas as pd
import random
import string

# 데이터프레임 생성
data = []

# 한글 글자 생성 함수
def generate_hangul(length):
    # 한글 유니코드 시작: 44032, 끝: 55203
    base_code = 44032
    last_code = 55203
    hanguls = []
    for _ in range(length):
        code = random.randint(base_code, last_code)
        hanguls.append(chr(code))
    return ''.join(hanguls)

# 데이터프레임에 랜덤한 한글 글자 추가
for _ in range(10):
    hangul_text = generate_hangul(10)
    data.append({'첫 번째 컬럼': hangul_text, '두 번째 컬럼': hangul_text +'국회의원' + hangul_text})

# 데이터프레임 생성
df = pd.DataFrame(data)

# 출력
print(df)

In [ ]:
# '국회의원'이 몇 번 나오는지 확인하여 새로운 컬럼 생성
split_column = df['두 번째 컬럼'].str.split('국회의원', expand=True)
num_occurrences = split_column.shape[1] - 1  # '국회의원'이 나오는 횟수

# '두 번째 컬럼'에 '국회의원' 앞의 단어 할당
df['두 번째 컬럼'] = split_column[0]

# 새로운 컬럼 생성하여 '국회의원' 뒤의 단어 할당
for i in range(num_occurrences):
    column_name = f'새로운 컬럼{i+1}'
    df[column_name] = split_column[i+1]

# 출력
print(df)